# ----------------------------------------------------------------------========
PROJECT'S TITLE
    DATA SCIENTIST PROJECT RELATED TO THE ANALYSIS OF HOSPITALIZATION
     DATA IN FRANCE DUE TO COVID-19.


PROJET/PURPOSE:
    The UDACITY DATA SCIENTIST CAPSTONE project consists in building
     a data science project of my choice with two deliverables:
      - A Github repository of my work;
      - A blog post written for the audience.

    Considering the rebound of COVID-19 cases in France these last
     weeks, I decided to analyze data of COVID-19 progress in France
     with figures directly reported by the hospitals via an
     institutional site.

    The blog post introduce the context of my study, the purpose of
     my study and a strategy to address it.
    To sum up, it will work on data sets to be able to forecast the 
     number of hospitalizations in a context of COVID-19 pandemy for
     the departement of Haute-Garonne (Region Occtinanie, France).

    This notebook allows:
     - Analyzing the data (data exploration and vizualization);
     - Propose a methodology for prepare the data set by 
        preprocessing data and deepen understanding of these data
        and finally kept the final data set;
     - Select the modeling method with related metrics and find out
        ways to improve this method.

The notebook here-below gathers my work for this purpose.

In [ ]:
# ANALYSIS - Data exploration - import libraries
# ----------------------------------------------------------------------========

import sys
import re
import time
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from IPython.display import SVG
from plotly.subplots import make_subplots
# import pyplot
# ERROR: Could not find a version that satisfies the requirement pyplot
#        (from versions: none)
#        No matching distribution found for pyplot

from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import neighbors
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import precision_recall_fscore_support
# from sklearn.metrics import jaccard_similarity_score
# ImportError: cannot import name 'jaccard_similarity_score' from
#  'sklearn.metrics'

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
# from sklearn.model_selection import GridSearchCV

from sklearn.metrics import RocCurveDisplay

# requires installing
# https://gtk-win.sourceforge.io/home/index.php/Main/Downloads

# ANALYSIS - Data exploration - describe in the input files
# ----------------------------------------------------------------------========

I've gathered several data files reporting various information related
 to the epidemic of COVID-19 in France from the institutional web site:
 https://www.data.gouv.fr/fr/datasets

The source web site indicates that data have been gathered along the
 time from various formats and contents.

All the files used are under the License "Open Data Commons Open
 Database License (ODbL)". 

I downloaded the following data files the 29 December 2022. All data are
 stored in Github of this project.


FILE # 1

- Initial name: "donnees-vaccination-par-tranche-dage-type-de-vaccin-et
                 -departement.csv"

- New name: "COVID-19_France_data_vaccin_population.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/donnees-vaccination-
               par-tranche-dage-type-de-vaccin-et-departement-region/"

- Last update: 13-Dec-2022

- License ID: 60bdce49abcc8f5dcb2fcb3b

- Brief description: Vaccination data by age bracket, type of vaccine
                     and department / region.

- Size: 146 Mb


FILE #2

- Initial name: "donnees-vaccination-par-pathologie.csv"

- New name: "COVID-19_France_data_vaccin_pathology.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/donnees-vaccination-
               par-pathologie-et-departement-region/"

- Last update: 13-Dec-2022

- License ID: 60bdce48e306a187d32ee2c1

- Brief description: Vaccination data per pathology and department
                     / region.

- Size: 1.2 Mb


FILE #3

- Initial name: "table-indicateurs-open-data-dep-2022-12-28-19h00.csv"

- New name: "COVID-19_France_data_epidemic_indicators.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/synthese-des-
               indicateurs-de-suivi-de-lepidemie-covid-19/"

- Last update: 28-Dec-2022

- License ID: 60190d00a7273a8100dd4d38

- Brief description: COVID-19 epidemic monitoring indicators summary

- Size: 11.2 Mb


FILE #4

- Initial name: "vacsi-dep-2022-12-28-19h00.csv"

- New name: "COVID-19_France_data_vaccin_indicators.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/r/735b0df8-51b4-
               4dd2-8a2d-8e46d77d60d8"

- Last update: 28-Dec-2022

- License ID: 6010206e7aa742eb447930f7

- Brief description: Data related to people vaccinated against COVID-19
                   per department.

- Size: 6.2 Mb


FILE #5

- Initial name: "vacsi-tot-dep-2022-12-28-19h00.csv"

- New name: "COVID-19_France_data_vaccin_indicators_tot.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/r/7969c06d-848e-
               40cf-9c3c-21b5bd5a874b"

- Last update: 28-Dec-2022

- License ID: 6010206e7aa742eb447930f7

- Brief description: Data related to cumulative people vaccinated 
                     against COVID-19 per department.

- Size: 10 Kb


COMMENT:

Data file were renamed to avoid being dependent of the date that may be
 notified in the file's name and to simplify and gather the designation
 of these files in the workspace.


All these files are known to be of type 'txt/csv'. They are fulfilled 
 in french such that I was required to open these files under Unicode
 UTF-8 format.


Let's explore these files through the notebook!

In [ ]:
# ANALYSIS - Data exploration - Workspace
# ----------------------------------------------------------------------========

# Purpose: Define the main directory of the data files: 'work_dir'
# Instruction: Modify value of 'work_dir' according to your workspace.
dir_1 = r"C:\Users\to202835\OneDrive - ATR\_exploitation\formation"
dir_2 = r"\2023\Udacity_DataScience\project_capstone_v1.4.0"
work_dir = dir_1 + dir_2

In [ ]:
# ANALYSIS - Data exploration - Workspace
# ----------------------------------------------------------------------========

# Purpose: Build a function that read input csv files
# Discussion: Though some of the used data files are also available in
#             json format, I prefered get all my data files in the same
#             format to ease the read.

def read_csv(filename,
             dtype_values={},
             filedir=work_dir,
             separator=',',
             encod_errors='ignore'):
    '''
    DESCRIPTION
        Read input csv files
    INPUT
        filename (string) is the name of the data file to read;
        dtype_values (dictionary) is the set of information to enforce
                     read of selected columns in a given format to 
                     possibly avoid warning message of Low memory:
                     default = none;
        filedir=work_dir (string) is the path of the data files 
                         directory.
                         default = work_dir (previously defined)
    OUTPUT
        df (pandas dataframe) is the dataframe resulting from the read
           of the input data file. If the read fails, it returns an 
           empty dataframe.
    '''

    # Build the complete file path
    filepath = "\\".join((filedir, filename))

    # trying read the csv data file
    try:
        df = pd.read_csv(filepath.replace('\\','/'),
             dtype=dtype_values, sep=separator,
             encoding_errors=encod_errors,
             low_memory=False)
        print("- Opening '{}'".format(filename))
    except:
        df = pd.DataFrame([])
        print("- Unable to open '{}'".format(filename))
        raise

    return df

In [ ]:
# ANALYSIS - Data exploration - access the data
# ----------------------------------------------------------------------========
"""
DISCUSSION
    Initial read of the file report an issue on format of column #2
     that causes a low memory warning. The issue was solved by setting
     the appropriate format of some columns with mixed types 
     (identified after the initial opening and analysis of the file) and
     by using separator ";" except for file #3
"""
# List of names of all input data files to work on
file_1 = r"COVID-19_France_data_vaccin_population.csv"
file_2 = r"COVID-19_France_data_vaccin_pathology.csv"
file_3 = r"COVID-19_France_data_epidemic_indicators.csv"
file_4 = r"COVID-19_France_data_vaccin_indicators.csv"
file_5 = r"COVID-19_France_data_vaccin_indicators_tot.csv"

# Define the labels that require a change of format
dtypes_1 = {'date_reference': str,
 'semaine_injection': str,
 'region_residence': str,
 'libelle_region': str,
 'departement_residence': str,
 'libelle_departement': str,
 'classe_age': str,
 'libelle_classe_age': str,
 'type_vaccin': str,
 'date': str}  # These columns have a mixed type of data
dtypes_2 = {}
dtypes_3 = {"date": str,
          "dep": str,
          "lib_dep": str,
          "lib_reg": str}  # These columns have a mixed type of data
dtypes_4 = {"dep": str}  # column 'dep' has mixed types. So I enforce
                         #  the format while reading the csv file to
                         #  avoid a low memory warning message.
dtypes_5 = {"dep": str}  # column 'dep' has mixed types. So I enforce
                         #  the format while reading the csv file to
                         #  avoid a low memory warning message.

files = dict()
files['f1'] = {'filename': file_1, 'dtypes':dtypes_1, 'sep': ';'}
files['f2'] = {'filename': file_2, 'dtypes':dtypes_2, 'sep': ';'}
files['f3'] = {'filename': file_3, 'dtypes':dtypes_3, 'sep': ''}
files['f4'] = {'filename': file_4, 'dtypes':dtypes_4, 'sep': ';'}
files['f5'] = {'filename': file_5, 'dtypes':dtypes_5, 'sep': ';'}

# Read the files and store data into dataframes
df = pd.DataFrame()
for f in files:
    print(f)
    if files[f]['sep'] == '':
        df = read_csv(files[f]['filename'], dtype_values=files[f]['dtypes'])
    else:
        df = read_csv(files[f]['filename'], dtype_values=files[f]['dtypes'], separator=files[f]['sep'])
    files[f]['df'] = df.copy(deep=True)
    df.head()
del df

In [ ]:
# ANALYSIS - Data exploration - access the data
# ----------------------------------------------------------------------========

# Get on view on file #1 data and push data into a dataframe
df1 = files['f1']['df'].copy(deep=True)
df1.head()

In [ ]:
# ANALYSIS - Data exploration - access the data
# ----------------------------------------------------------------------========

# Get on view on file #2 data and push data into a dataframe
df2 = files['f2']['df'].copy(deep=True)
df2.head()

In [ ]:
# ANALYSIS - Data exploration - access the data
# ----------------------------------------------------------------------========

# Get on view on file #3 data and push data into a dataframe
df3 = files['f3']['df'].copy(deep=True)
df3.head()

In [ ]:
# ANALYSIS - Data exploration - access the data
# ----------------------------------------------------------------------========

# Get on view on file #4 data and push data into a dataframe
df4 = files['f4']['df'].copy(deep=True)
df4.head()

In [ ]:
# ANALYSIS - Data exploration - access the data
# ----------------------------------------------------------------------========

# Get on view on file #5 data and push data into a dataframe
df5 = files['f5']['df'].copy(deep=True)
df5.head()

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========

# Build a function to analysis the type and availability of valid data.

def display_columns_info(df):
    '''
    DESCRIPTION
        Display some information per column related to the data set
    INPUT
        df is the dataframe
    OUTPUT
        nil (only display)    
    '''
    
    # Measure length of the column's title only for presentation purpose
    max_len = 0
    cols = np.array(df.columns)
    for c in cols:
        if len(c) > max_len:
            max_len = len(c)
    size = max_len + 2  # I will create a regulat length for label field.

    # Display information per column        
    field_1 = "column label" + ' '*(size - 11)
    field_size = 10
    field_2 = "      size "
    field_3 = "      type "
    field_4 = "   nb uniq  rate uniq "
    field_5 = "    nb nan   rate nan "
    field_6 = "    nb inf   rate inf "
    field_7 = "     other "
    # Introducing the results
    m = field_1 + field_2 + field_3 + field_4 + field_5 + field_6 + field_7
    print(m)

    # Then run along the columns to perform and give the analysis per column.
    for col in cols:
        # column's size
        col_size = df[col].shape[0]
        # get column's type        
        col_type = df[col].dtypes
        cot_typ = str(col_type)
        # get number and rate of unique values
        values_uniq = np.array(pd.unique(df[col]))
        nb_uniq = len(values_uniq)
        rate_uniq = 100 * nb_uniq / col_size
        # get number and rate of nan values
        nb_nan = df[col].isna().sum()
        rate_nan = 100 * nb_nan / col_size
        # get number and rate of infinite values
        nb_inf = 0
        mixed_types = []
        # Run along values of the column
        for value in np.array(df[col].values):
            # Count infinite value
            if (value == np.inf) or (value == -np.inf):
                nb_inf += 1
            # Check consistency of the value type with its column
            if " " in str(type(value)):
                val_type = str(type(value)).split("'")[1]
            else:
                if "." in str(type(value)):
                    val_type = str(type(value)).split(".")[1][:-1]
                else:
                    val_type = str(type(value))
            if (val_type != cot_typ):
                if val_type not in mixed_types:
                    mixed_types.append(val_type)

        # Compute the rate of infinite values in the column
        rate_inf = 100 * nb_inf / col_size

        # Report mixed types if any
        msg_mixed_types = ''
        if len(mixed_types) > 1:
            msg_mixed_types = '\tMixed types: ' + str(mixed_types)

        # Check among 'date' values that they are all strings with a length
        #  of 10.
        mem_unexp_dates = []
        msg_unexp_dates = ''
        if col == 'date':
            for value in values_uniq:
                if (not isinstance(value, str)) or (len(value) != 10):
                    mem_unexp_dates.append(value)
            if len(mem_unexp_dates) > 0:
                msg_unexp_dates = '\tWrong date format: ' + str(mem_unexp_dates)

        # Display result
        dec = 2
        print('{}{} {}{} {}{} {}{}{}{} % {}{}{}{} % {}{}{}{} % {} {}'.format(
            col, ' '*(size - len(col)),
            ' ' * (field_size - len(str(col_size))), col_size,
            ' ' * (field_size - len(str(col_type))), col_type,
            ' ' * (field_size - len(str(nb_uniq))), nb_uniq,
            ' ' * (field_size -1 - len(str(round(rate_uniq, dec)))), round(rate_uniq, dec),
            ' ' * (field_size - len(str(nb_nan))), nb_nan,
            ' ' * (field_size -1 - len(str(round(rate_nan, dec)))), round(rate_nan, dec),
            ' ' * (field_size - len(str(nb_inf))), nb_inf,
            ' ' * (field_size -1 - len(str(round(rate_inf, dec)))), round(rate_inf, dec),
            msg_unexp_dates,
            msg_mixed_types)
            )

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# Vizualize information on file 1
display_columns_info(df1)

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# According to the result, I will remove columns almost empty
#  (at least 50% of nan):
remove_list1 = ['effectif_cumu_1_inj', 'effectif_cumu_termine',
         'taux_cumu_1_inj', 'taux_cumu_termine', 'effectif_1_inj',
         'effectif_termine', 'taux_1_inj', 'taux_termine', 
         'effectif_rappel', 'effectif_cumu_rappel', 
         'effectif_rappel_parmi_eligible',
         'effectif_eligible_au_rappel', 'taux_rappel',
         'taux_cumu_rappel', 'taux_cumu_rappeleli']

# Refer to the blog post for other comments

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# Vizualize information on file 2
display_columns_info(df2)

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# Vizualize information on file 3
display_columns_info(df3)

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# According to the result, remove these columns with a lot of missing
#  data.
remove_list3 = ['R', 'cv_dose1']

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# Vizualize information on file 4
display_columns_info(df4)

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# According to the result, rename column 'jour' as 'date'
renaming_4_dict = {'jour': 'date'}

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# Vizualize information on file 5
display_columns_info(df5)

In [ ]:
# ANALYSIS - Data exploration - basic analysis of the data sets
# ----------------------------------------------------------------------========
# According to the result, rename column 'jour' as 'date' (idem for file #4)
renaming_5_dict = {'jour': 'date'}

In [ ]:
df4.rename(renaming_4_dict, axis=1, inplace=True)
df5.rename(renaming_5_dict, axis=1, inplace=True)
# to be done before merging because it's a label to merge with
# and before time series analysis, because it's based on 'date'.

In [ ]:
# ANALYSIS - Data exploration - Time series analysis
# ----------------------------------------------------------------------========

# Basically, I want to know how many unique dates owns every dataframe:
df_list = [df1, df2, df3, df4, df5]
print('Number of unique values of date per input data file:')
for i, df in enumerate(df_list):
    print("File #", i+1, '\t', len(np.unique(np.array(df['date']))))
# note: Applying the function 'display_columns_info' give me already
#  the same result but it allows to summarize it here.

# Result: I see that merging all files would give me only one date.
#         So I need to identify clearly the covering time period of
#         every files to see which ones I could merge.
#         A priori, I would keep only files 3 and 4, let's check that.
#         Files 2 and 5 return only one date: ['2022-12-04']

In [ ]:
# ANALYSIS - Data exploration - Time series analysis
# ----------------------------------------------------------------------========

# Build a generic function for displaying time series

def plot_time_series(df, label: str, departments: list):
    '''
    DESCRIPTION
        Plot a line chart of figures for the given label and the 
         selected list of departments.
    INPUT
        df is the pandas dataframe;
        label (string) is the name of the column for which I want a
              plot;
        departments (list) is the list departments for which I want
                    a plot.
    OUTPUT
        nil (only display)
    '''

    # If not already done, I group df by department and timestamp
    # df.groupby(['dep', 'timestamp'])  # If no dummy applied
    df.groupby(['dep', 'date'])  # If no dummy applied

    # get the uniq list of the selected departments, just in case of...
    departments = np.unique(departments)  # just in case of 
    
    df_plot_list = []  # list to store data for plotting of every
                       #  selected departments.
    # Run along the list of departments
    for dep in departments:
        # Get the values of the label along the time
        df_dep = pd.DataFrame([])
        df_dep = df[df['dep'] == dep]

        # Get the time for this dep:
        x_val = np.array(df_dep['date'])
        
        # get the values of the label for this dep
        y_val = np.array(df_dep[label])

        # Create a dataframe for the plot
        df_plot = pd.DataFrame(dict(x = x_val, y = y_val))

        # store the data for plotting
        df_plot_list.append(df_plot)

    # Plotting
    # declare the figure
    fig = go.Figure()

    # Run along the list of dataframes containing plotting data
    for i, df_plot in enumerate(df_plot_list):
        # Add a line of result per dataframe
        fig.add_trace(go.Scatter(x=np.array(df_plot['x']),
                                y=np.array(df_plot['y']),
                                mode='lines+markers',
                                name='Dept. ' + str(departments[i])
                                )
                    )
    # Define the context of the plot
    titre = df_dict_labels[label] + "\n(per department)"
    fig.update_layout(title=titre,
                    xaxis_title="Time",
                    yaxis_title=df_dict_labels[label],
                    legend=dict(traceorder='reversed'))
    fig.show()

In [ ]:
# ANALYSIS - Data exploration - Time series analysis
# ----------------------------------------------------------------------========

# I build a function to format x/y values for plotting lines

def get_df_dates(dfs, label='timestamp'):
    '''
    DESCRIPTION
        return dataframes with min/max of the column label in abscisse
         and index in ordonate. For visualization purpose
    INPUT
        dfs (list) is a list of dataframes where I will search for  
            values in the column named label;
        label (string) is the name of the column where I will lokk for 
              timestamp values; default = 'timestamp'.
    OUTPUT
        dfs_return (list) is a list of dataframes such that every
                   dataframe is defined like:
                   {'x' = [values of ‘dates’ from the dataframe] }
                   {'y' = [position of the dataframe in the list] }
    '''
    dfs_return = []

    # Run along the list of dataframes
    for i, df in enumerate(dfs):
        '''
        # sort timestamp by ascending order
        df.sort_values(by='timestamp', ascending=True, inplace=True)
        # get min and max timestamp values given by the dataframe
        min_ts, max_ts = df['timestamp'].min(), df['timestamp'].max()
        # get date (str) equivalent for these min/max timestamps
        min_date = get_df_related_value(df, 'timestamp', 'date', min_ts)
        max_date = get_df_related_value(df, 'timestamp', 'date', max_ts)
        # build a dataframe with min/max dates and index related to 
        #  the dataframe.
        df_ts = pd.DataFrame(dict(
                x = [min_date, max_date],
                y = [i+1, i+1]
        ))
        '''
        # get X/Y values
        x_values = df['date'].tolist()
        y_values = [i+1] * len(x_values)
        # format X/Y for ploting
        df_ts = pd.DataFrame(dict(
                x = x_values,
                y = y_values
        ))
        
        # Store the dataframe into the the list of dataframes
        dfs_return.append(df_ts)

    return dfs_return

In [ ]:
# ANALYSIS - Data exploration - Time series analysis
# ----------------------------------------------------------------------========

# I build a function to format x/y values for plotting lines

def plot_df_time_periods(dfs):
    '''
    DESCRIPTION
        Plot a line chart showing the time period of every input 
         dataframe.
    INPUT
        dfs (list) is a list of dataframes such that every
            dataframe is defined like:
            {'x' = [values of ‘dates’ from the dataframe] ] }
            {'y' = [position of the dataframe in the list] }
    OUTPUT
        nil (only display)
    '''
    # declare the figure
    fig = go.Figure()

    # Run along the list of dataframes containing plotting data
    for i, df in enumerate(dfs):
        # Add a line of result per dataframe
        fig.add_trace(go.Scatter(x=np.array(df['x']),
                                 y=np.array(df['y']),
                                 mode='lines+markers',
                                 name='file' + str(i+1)
                                )
                    )
    # Define the context of the plot
    titre = 'Difference of time period covered per data file'
    fig.update_layout(title=titre,
                    xaxis_title="time line",
                    yaxis_title="File#",
                    legend=dict(traceorder='reversed'))
    fig.update_yaxes(range=[0.5, len(dfs)+0.5])
    fig.show()

In [ ]:
# ANALYSIS - Data exploration & Vizualization - Time series analysis
# ----------------------------------------------------------------------========
# plot time period for every input dataframe
dfs = get_df_dates(df_list)  # Get data for plotting
plot_df_time_periods(dfs)  # Plot the time periods

# ANALYSIS - Data exploration & Vizualization - Time series analysis
# ----------------------------------------------------------------------========
# According to the times epriod s showed here-above and the cound of
#  dates per files, I will keep only data from files #3 and #4 for 
#  merging and further works.

In [ ]:
# ANALYSIS - Data exploration & Vizualization - labels overlapping
# ----------------------------------------------------------------------========
# before merging dataframes, I'll define how labels will be used merging
#  dataframes and if there is no overlapping of labels that could create
# some conflicts.

df3_labels = df3.columns.values
df4_labels = df4.columns.values

# I search among labels of df3 which are also present in df4
for label in df3_labels:
    if label in df4_labels:
        print('df3_labels[', label, '] also in df4_labels')

# I search among labels of df4 which are also present in df3
for label in df4_labels:
    if label in df3_labels:
        print('df4_labels[', label, '] also in df3_labels')

# Conclusion only labels 'dep' and 'date' are common in both files

In [ ]:
# METHODOLOGY - Data preprocessing - Files data merging
# ----------------------------------------------------------------------========

# Build a function that provide the list of common values through
#  several dataframes for a given column's label.

'''Discussion
It may take a long time so there is certainly means to re-factorize 
 this part of the code.
'''

def get_common_values(dfs, label):
    '''
    DESCRIPTION
        get common values on the selectec label through all given 
         dataframes
    INPUT
        dfs (list) is the list of all dataframes.
        label (string) is the name of the column for which I want
         find common values.
    OUTPUT
        common_values is a numpy array of the common values
    '''
    common_values, all_values = [], []

    # run along every dataframe to find all values through all
    #  dataframes.
    for df in dfs:
        # get an array of values of the selected label
        values = np.unique(np.array(df[label]))
        # run along these values 
        for value in values:
            # if the value is not in the list of all values, I add it 
            #  to this list
            if value not in all_values:
                all_values.append(value)
    # all_values contains the list of all values of the column label 
    #  contained in all dataframes.

    # Run along all values to find only common values through all
    #  dataframes.
    for value in all_values:
        cnt = 0  # count the number of occurrence of this value
        for df in dfs:
            values = np.unique(np.array(df[label]))
            if value in values:
                cnt += 1
        if cnt == len(dfs):
            common_values.append(value)
    
    return np.array(common_values)


# Test function get_common_values
df_test1 = pd.DataFrame({'date': ['2020-05-12', '2020-06-13', '2021-07-06', '2021-08-01', '2022-01-22', '2022-02-23']})
df_test2 = pd.DataFrame({'date': ['2020-05-13', '2020-06-13', '2021-07-06', '2021-08-01', '2022-01-22', '2022-02-23']})
df_test3 = pd.DataFrame({'date': ['2020-05-13', '2020-06-13', '2021-07-06', '2021-08-01', '2022-01-22', '2022-02-24']})
toto = get_common_values([df_test1, df_test2, df_test3], 'date')
print('computed {}\nexpected {}'.format(toto, np.array(['2020-06-13', '2021-07-06', '2021-08-01', '2022-01-22'])))
del df_test1, df_test2, df_test3

In [ ]:
# METHODOLOGY - Data preprocessing - Files data merging
# ----------------------------------------------------------------------========

# Merge the dataframes with 'date' and 'dep' as common columns.
df = pd.merge(df3, df4, on=['date', 'dep'], how='inner')
print(df3.shape, "+", df4.shape, "=", df.shape)

In [ ]:
# A step back
# ANALYSIS - Data exploration and vizualization - distribution of data
# ----------------------------------------------------------------------========
# Since I would work on different level of area (department, region,
#  France), I would have a view on the distribution of data per 
#  department and region
# Haute-Garonne = 31

# Count number of rows (date) per department
# create a dictionary of this counting:
# key = dept id, value = count of rows per dept.
df_sort_dep2_dict = (df.groupby(['dep'])['dep'].count()).to_dict()

# push keys and values in list for further plotting
dep_ids, dep_data_qty = [], []
qty_max = 0
for dep_id in df_sort_dep2_dict:
    dep_ids.append(dep_id)
    qty = df_sort_dep2_dict[dep_id]
    dep_data_qty.append(qty)
    if qty > qty_max:
        qty_max = qty
print('Maximum:', qty_max)

# plot quantity of data per department
titre = "Quantity of data per French department"
fig = go.Figure(
    data=[go.Bar(x=dep_ids, y=dep_data_qty)],
    layout=go.Layout(
        title=go.layout.Title(text=titre),
        xaxis_title="dept id",
        yaxis_title="Nb of dates with data"
    )
)
fig.show()

In [ ]:
# ANALYSIS - Data exploration and vizualization - distribution of data
# ----------------------------------------------------------------------========
# Count number of rows (date) per region
# Occitanie = 76

# Count number of rows (date) per region

# create a dictionary of this counting:
# key = region id, value = count of rows per region.
df_sort_reg2_dict = (df.groupby(['reg'])['reg'].count()).to_dict()

# push keys and values in list for further plotting
reg_ids, reg_data_qty = [], []
max_qty = 0
for reg_id in df_sort_reg2_dict:
    reg_ids.append(reg_id)
    qty = df_sort_reg2_dict[reg_id]
    reg_data_qty.append(qty)
    if qty > qty_max:
        qty_max = qty
print('Maximum:', qty_max)

# plot quantity of data per region
fig = go.Figure(
    data=[go.Bar(x=reg_ids, y=reg_data_qty)],
    layout=go.Layout(
        title=go.layout.Title(text="Quantity of data per French region"),
        xaxis_title="reg id",
        yaxis_title="Nb of dates with data"
    )
)
fig.show()

In [ ]:
# METHODOLOGY - Data preprocessing - Files data merging
# ----------------------------------------------------------------------========
# See above (before checking of the data distribution)
# let's have a view on the result of the analysis of the merged data set
display_columns_info(df)

In [ ]:
# METHODOLOGY - Data preprocessing - Missing values
# ----------------------------------------------------------------------========
# Remove columns with too many missing values in accordance with
#  observations raised during the data exploration and raised here-above

# df.drop(columns=remove_list1, inplace=True)  # from df1 (not used)
df.drop(columns=remove_list3, inplace=True)  # from df3

In [ ]:
# METHODOLOGY - Data preprocessing - labels renaming
# ----------------------------------------------------------------------========
# Rename columns with too many missing values in accordance with
#  observations raised during the data exploration 

# df.rename(renaming_4_dict, axis=1, inplace=True)  # from df4
# to be done before merging because it's a label to merge with
# and before the time series analysis which is based on 'date'.

#df.rename(renaming_5_dict, axis=1, inplace=True)  # from df5 (not used)

In [ ]:
# METHODOLOGY - Data preprocessing - Data format
# ----------------------------------------------------------------------========

# dep has mix format values because of the presence of the 2 departments
#  in Corse: 2A and 2B; for ease use of the dep values, I want all these
#  values in integer format; so I will convert these both hybrid values
#  into intergers

# create a function that allows replacing str by int values

def replace_str_id(df, label):
    '''
    DESCRIPTION

    INPUT
        df is the pandas dataframe
        label (string) is the label of the column to process
              'dep' or 'region'.
    OUTPUT
        df is the input pandas dataframe with string values of label
        replace by int values.

    comment:
        think about specific Corse departments 2A and 2B
        that I propose to replace by 201 and 202.

    '''
    dico = {'2A': 201, '2B': 202}

    # get unique values of the label
    values = np.array(df[label].values)

    convert = []
    # Run along the values for conversion from str to int
    for value in values:
        # It try to refers to the dictionary for Corse's deps
        try:
            # basically, I try to convert str to int
            c = int(value)
        except:
            # when it fails,
            if value in dico:
                c = dico[value]
            else:
                # otherwise I apply value zero
                c = 0
        '''
        if (value in dico_keys):
            c = dico[value]
        else:
        '''
        convert.append(c)

    df[label] = np.array(convert)
    df[label] = df[label].astype(int)  # Ensure format for the columns.
                                       # Do not put before otherwise,
                                       #  it fails to convert 2A and 
                                       #  2B and display and warning

    return df

# check the function
df_test = pd.DataFrame({'dep': np.array(['01', 21, '2A', '2B', '196'])})
result = replace_str_id(df_test, 'dep')
solution = [1, 21, 201, 202, 196]
print('computed: {}\nexpected: {}'.format(np.array(result['dep']),
                                          solution))

# I convert 'dep' and 'reg' to int values
df = replace_str_id(df, 'dep')
df = replace_str_id(df, 'reg')

In [ ]:
# METHODOLOGY - Data preprocessing - Data format
# ----------------------------------------------------------------------========
# In addition, I should remove the textual name of the departments to
#  keep only values into the dataframe.
# Prior removing these names, I will built a dictionary with these names
#  and their number to recall these names for further display if needed.

# Create a dictionary: key = dept id, value = dept name
dep_nums = df['dep'].values
dep_names = df['lib_dep'].values
dep_num_to_name_dict = dict()
for i, num in enumerate (dep_nums):
    if num not in dep_num_to_name_dict:
        dep_num_to_name_dict[num] = dep_names[i]
print('dict of dep:\n', dep_num_to_name_dict)

# Idem with regions
reg_nums = df['reg'].values
reg_names = df['lib_reg'].values
reg_num_to_name_dict = dict()
for i, num in enumerate (reg_nums):
    if num not in reg_num_to_name_dict:
        reg_num_to_name_dict[num] = reg_names[i]
print('dict of reg:\n', reg_num_to_name_dict)

In [ ]:
# METHODOLOGY - Data preprocessing - Data format
# ----------------------------------------------------------------------========
# Build Department & Region dictionaries

# I build a function dedicated to the identification of a name of
#  the department according to its id. 

def get_dep_name(dep_id):
    '''
    DESCRIPTION
        give the name of a department in accordance with its id
    INPUT
        dep_id is the id (int) of the department from
               which we want the name.
    OUTPUT
        dep_name is the name (str) of the department related to the
                 given id.
    '''
    try:
        name = dep_num_to_name_dict[dep_id]
    except:
        name = 'unknown'
    return name

# Check the function with various formats

t_int = get_dep_name(45)
t_cor = get_dep_name(201)
t_unk = get_dep_name(600)
print('Loiret: {}'.format(t_int))
print('Corse-du-Sud: {}'.format(t_cor))
print('unknown: {}'.format(t_unk))

# Then I build a function dedicated to the identification of a name of
#  the region according to its id. 
def get_reg_name(reg_id):
    '''
    DESCRIPTION
        give the name of a region in accordance with its id
    INPUT
        reg_id is the id (str, int or float) of the region from which
               we want the name.
    OUTPUT
        reg_name is the name (str) of the region related to the given
                 id.
    '''
    try:
        name = reg_num_to_name_dict[reg_id]
    except:
        name = 'unknown'
    return name

# Check the function with various formats
t_int = get_reg_name(93)
t_unk = get_reg_name(604)
solution = "Provence-Alpes-Côte d'Azur"
print("{}: {} - unknown: {}".format(solution, t_int, t_unk))

In [ ]:
# METHODOLOGY - Data preprocessing - Data format
# ----------------------------------------------------------------------========
# Then by extension, I build a function to provide the name of
#  departments from a list of their ids.

def get_dep_names(dep_ids: list):
    '''
    DESCRIPTION
        give a list of department names according to a list of
         department ids.
    INPUT
        dep_ids is the list of department id
    OUTPUT
        dep_names is the list of department names
    '''
    dep_names = list()
    for dep_id in dep_ids:
        dep_names.append(get_dep_name(dep_id))

    return dep_names

# Check the function
list_dep_ids = [45, 21]
solution = ['Loiret', "Côte-d'Or"]
print('expected: {} vs computed: {}'.format(solution,
                                        get_dep_names(list_dep_ids)))


# In the same way, I build a function to provide the name of region from
#  a list of their ids.
def get_reg_names(reg_ids: list):
    '''
    DESCRIPTION
        give a list of region names according to a list of region ids
    INPUT
        reg_ids is the list of region id
    OUTPUT
        reg_names is the list of region names
    '''
    reg_names = list()
    for reg_id in reg_ids:
        reg_names.append(get_reg_name(reg_id))

    return reg_names

#  Check the function
list_reg_ids = [76, 27]
solution = ['Occitanie', 'Bourgogne et Franche-Comté']
print('expected: {} vs computed: {}'.format(solution,
                                        get_reg_names(list_reg_ids)))

In [ ]:
# METHODOLOGY - Data preprocessing - Data format
# ----------------------------------------------------------------------========
# Finally I can remove labels of department and region since I have
#  their dictionaries.
size_before = df.shape
df.drop(columns=['lib_dep', 'lib_reg'], inplace=True)

# check result of the action:
title = "df's size has changed from"
print("{} {} to {}:".format(title, size_before, df.shape))

In [ ]:
# METHODOLOGY - Data preprocessing - Labels description
# ----------------------------------------------------------------------========
# Create a dictionary with a shot description of every label for further
#  use during visualization of results.
df_dict_labels = {
    'timestamp': 'number of days passed from start of year 0 A.D.',
    'dep': 'Dept.',
    'reg': 'Region',
    'tx_pos': 'Positivity rate',
    'tx_incid': 'Incidence rate',
    'TO': 'Hospital occupancy rate',
    'hosp': 'Hospitalized People',
    'rea': 'People in resuscitation/Intensice care',
    'rad': 'Cumulative hospitalized people back to home',
    'dchosp': 'Death at hospital',
    'reg_rea': 'unknown',
    'incid_hosp': 'Nes hospitalization in last 24h',
    'incid_rea': 'New people in resuscitation/Intensice care in last 24h',
    'incid_rad': 'New people hospitalized back to home in last 24h',
    'incid_dchosp': 'New death in last 24h',
    'reg_incid_rea': 'unknown',
    'pos': 'Positive case (J-3)',
    'pos_7j': 'Positive case on a week',
    'n_dose1': 'Vaccinated with 1 dose',
    'n_complet': 'Vaccinated with 2 doses',
    'n_rappel': 'Vaccinated with 1 booster dose',
    'n_2_rappel': 'Vaccinated with 2 booster doses',
    'n_rappel_biv': 'Vaccinated with 1 booster dose (adj. Omicron)',
    'n_3_rappel': 'Vaccinated with 3 booster doses',
    'n_cum_dose1': 'Cumulative vaccinated with 1 dose',
    'n_cum_complet': 'Cumulative vaccinated with 2 doses',
    'n_cum_rappel': 'Cumulative vaccinated with 1 booster dose',
    'n_cum_2_rappel': 'Cumulative vaccinated with 2 Booster doses',
    'n_cum_rappel_biv': 'Cumulative vaccinated with 1 booster dose (adj. Omicron)',
    'n_cum_3_rappel': 'Cumulative vaccinated with 3 Booster doses',
    'couv_dose1': 'Vaccination coverage with 1 dose',
    'couv_complet': 'Vaccination coverage with 2 doses',
    'couv_rappel': 'Vaccination coverage with 1 booster dose',
    'couv_2_rappel': 'Vaccination coverage with 2 booster doses',
    'couv_rappel_biv':'Vaccination coverage with 1 booster dose (adj. Omicron)',
    'couv_3_rappel': 'Vaccination coverage with 3 booster doses'
}

In [ ]:
# METHODOLOGY - Data preprocessing - Time format
# ----------------------------------------------------------------------========
# Add timestamp column and remove the column with dates

# Create a kind of calendar that provide the number of cumumlated passed
#  days at beginning of every month since beginning of the year.

def get_cumul_days():
    '''
    DESCRIPTION
        Return a list to ease count, month per month, passed day from
         the beginning of the year.
    INPUT
        nil
    OUTPUT
        cnt_day_at_start_month is a list that gives the nb of past days
         from beginning of the year for every month.
    '''
    cnt_day_at_start_month = []

    day_by_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    cnt = 0

    for i in day_by_month:
        cnt += i 
        cnt_day_at_start_month.append(cnt)

    return cnt_day_at_start_month

# Chech the function
calendar = get_cumul_days()
print('calender:', calendar)


# Then I use this calendar to build a function that will provide the
#  number of passed days since beginning of the year to beginning of
#  the selected month.

def cnt_day_beg_month(m, calendar):
    '''
    DESCRIPTION
        get the number of past days from the beginning of year and at 
         beginning of the selected month.
    INPUT
        m is the selected month (str of int, int or float)
        calendar is a list that gives the nb of past days from
                 beginning of the year for every month
    OUTPUT
        cnt is the count of days (int) from beginning of the year at
         start of the selected month.
    '''
    cnt = calendar[int(m)-1]
    return cnt

# Check the function
# 31 days passed at beginning of february.
result = cnt_day_beg_month(2, calendar)
print('result: {} vs expected: {}'.format(result, 31))


# Finally, I compute a timestamp from the dates as the number of days
#  passed  from 0 to the selected day such as for instances:
# 2022-06-24 => 2022 * nb_days_passed_to_this_year from year 0
#               + nb_passed_days_from_begin_of_the_year_to_begin_of
#                 this_month
#               + 24 (nb of days passed since beginning of the month)
#               = 2022 * 365.24219 + 151 + 24 = 738695
# 0000-01-01 => 0 + 0 + 1 = 1


def get_timestamp_array(dates_str):
    '''
    DESCRIPTION
        Convert an np.array of dates in string format
        to a timestamp in number of days.
    INPUT
        dates_str is a np.array of dates YYYY-MM-DD in string format
    OUTPUT
        timestamp is np.array with data converted in number of days(int)
    '''
    timestamp = []
    j = 0
    for d in dates_str:
        try:
            y = float(d[:4])
            m = float(d[5:7])
            d = float(d[8:10])
            # convert the data in number of days
            cnt_days_for_months = float(cnt_day_beg_month(m, calendar))
            ts = y * 365.24219 + cnt_days_for_months + d
        except:
            ts = 0.0
        ts_r = round(ts, 0)
        timestamp.append(int(ts_r))
        j += 1
    return np.array(timestamp)

# Check the function
test_dates = np.array(['2022-06-24', '0000-01-01'])
timestamp_arr = get_timestamp_array(test_dates)
print('expected: {} vs computed {}'.format([738695, 1], timestamp_arr))


# Add a timestamp column in df
# It will hep for displaying data in time series and find most recent
#  values per department.
df['timestamp'] = get_timestamp_array(np.array(df['date']))
# update the labels dictionary
df_dict_labels['timestamp'] = 'Timestamp'
# As a reminder, I've decided to keep the column 'date' for avoid 
#  need to rebuild the date value in string format from the timestamp
#  when needed.

# remove the column 'date' (str):
df_ = df.copy(deep=True)
df.drop(columns='date', inplace=True)

In [ ]:
# METHODOLOGY - Data preprocessing - view
# ----------------------------------------------------------------------========
# Let's have a view on the result of the operations
display_columns_info(df)

In [ ]:
# METHODOLOGY - Data preprocessing - view
# ----------------------------------------------------------------------========
# Remove residual missing values

size_before = df.shape
# I remove all rows containing at least one nan
df.dropna(axis='index', how='any', inplace=True)

# check result of the action:
title = "df's size has changed from"
print("{} {} to {}:".format(title, size_before, df.shape))

# Check result of the action
display_columns_info(df)

In [ ]:
# METHODOLOGY - Data understanding - Distribution of targets' values
# ----------------------------------------------------------------------========
# With the purpose of modeling behvaiour of 'hosp' and 'rea' in function
#  of the other parameters, I can plot this relation prior modeling it.

# For that purpose, I will consider three sets of data:
#  - Data for France
#  - Data for the Region Occitanie #76
#  - Data for the department Haute-Garonne #31

# One dataframe for France
df_Fr = df.copy(deep=True)
# remove departments 'dep', 'reg' and time 'timestamp'
df_Fr = df_Fr.drop(columns=['dep', 'reg', 'timestamp'])
print('dataframe France:', df_Fr.shape)

# One dataframe for region Occintanie
df_R76 = df.copy(deep=True)
# keep only data for region Occitanie #76
df_R76 = df_R76.loc[df_R76['reg'] == 76]
# remove departments 'dep', 'reg' and time 'timestamp'
df_R76 = df_R76.drop(columns=['dep', 'reg', 'timestamp'])
print('dataframe Occitanie:', df_R76.shape)

# One dataframe for department Haute-Garonne
df_D31 = df.copy(deep=True)
# keep only data for department Haute-Garonne 31
df_D31 = df_D31.loc[df_D31['dep'] == 31]
# remove departments 'dep', 'reg' and time 'timestamp'
df_D31 = df_D31.drop(columns=['dep', 'reg', 'timestamp'])
print('dataframe haute-Garonne:', df_D31.shape)


# Let's have a view on the value distribution of both targets: 'hosp' and 'rea)
# source: https://moncoachdata.com/blog/regression-lineaire-sur-un-cas-reel/
# Plot of the target label 'hosp'
sns.displot(data=df_Fr,x='hosp', bins=30, kde=True).set(title='Values distribution on France')
sns.displot(data=df_R76,x='hosp', bins=30, kde=True).set(title="Values distribution on Region Occitanie")
sns.displot(data=df_D31,x='hosp', bins=30, kde=True).set(title="Values distribution on department Haute-Garonne")

# Plot of the target label 'rea'
sns.displot(data=df_Fr,x='rea', bins=30, kde=True).set(title='Values distribution on France')
sns.displot(data=df_R76,x='rea', bins=30, kde=True).set(title="Values distribution on Region Occitanie")
sns.displot(data=df_D31,x='rea', bins=30, kde=True).set(title="Values distribution on department Haute-Garonne")

In [ ]:
# METHODOLOGY - Data understanding - Correlation with targets
# ----------------------------------------------------------------------========
# This step consists in plotting targets in function of other labels.

# So I start be defining a function to plot these values
def plot_x_y(df, x_label_list, y_label, subtitle):
    """
    DESCRIPTION
        Plot Y=f(X) for one Y and a list of X
    INPUTS
        df  is the dataframe to read for plotting
             a list of labels in x in relation with
             a unique label in y
        x_label_list  is a list of labels (string) for
                       whom I want to plot values in x.
        y_label  is the label (string) for whom I want
                  to plot values in y.
    OUTPUTS
        nil (plot plt)
    """
    plt.figure(figsize=(20, 5))

    # Plot a graph for every label of the x list
    for i, col in enumerate(x_label_list):
        plt.subplot(1, len(x_label_list) , i+1)
        x = df[col]
        y = df[y_label]
        plt.scatter(x, y, marker='o')
        plt.title(y_label + '=fn(' + col + ') - Area: ' + subtitle)
        plt.xlabel(col)
        plt.ylabel(y_label)


# Get the list of all labels available
labels_list = df_Fr.columns.to_list()
# df_Fr contains df's labels without 'dep', 'reg' and 'timestamp': ok

# Define a function to remove a label from a list of labels
def remove_label_from_list(input_list, label_to_remove):
    output_list = []
    for label in input_list:
        if label != label_to_remove:
            output_list.append(label)
    return output_list

# Remove labels of both targets 'hosp' and 'rea' from the list of labels
hosp_list = remove_label_from_list(labels_list, 'hosp')
liste = remove_label_from_list(hosp_list, 'rea')

In [ ]:
# METHODOLOGY - Data understanding - Correlation between targets
# ----------------------------------------------------------------------========
# plot 'rea' in function of 'hosp' at level of France
plot_x_y(df_Fr, ['hosp'], 'rea', 'France')

In [ ]:
# METHODOLOGY - Data understanding - Correlation between targets
# ----------------------------------------------------------------------========
# plot 'rea' in function of 'hosp' at level of the Region
plot_x_y(df_R76, ['hosp'], 'rea', 'Region Occitanie (#76)')

In [ ]:
# METHODOLOGY - Data understanding - Correlation between targets
# ----------------------------------------------------------------------========
# plot 'rea' in function of 'hosp' at level of the department
plot_x_y(df_D31, ['hosp'], 'rea', 'Department Haute-Garonne (#31)')

In [ ]:
# METHODOLOGY - Data understanding - Correlation with targets
# ----------------------------------------------------------------------========
# plot labels for target 'hosp' at level of France
start = 0
end = start + 6
plot_x_y(df_Fr, liste[start:end], 'hosp', 'France')

In [ ]:
start = end
end = start + 6
plot_x_y(df_Fr, liste[start:end], 'hosp', 'France')

In [ ]:
start = end
end = start + 6
plot_x_y(df_Fr, liste[start:end], 'hosp', 'France')

In [ ]:
start = end
end = start + 6
plot_x_y(df_Fr, liste[start:end], 'hosp', 'France')

In [ ]:
start = end
end = start + 7
plot_x_y(df_Fr, liste[start:end], 'hosp', 'France')

In [ ]:
# METHODOLOGY - Data understanding - Correlation with targets
# ----------------------------------------------------------------------========
# plot labels for target 'hosp' at level of Region Occitanie
start = 0
end = start + 6
plot_x_y(df_R76, liste[start:end], 'hosp', 'Reg.76')

In [ ]:
start = end
end = start + 6
plot_x_y(df_R76, liste[start:end], 'hosp', 'Reg.76')

In [ ]:
start = end
end = start + 6
plot_x_y(df_R76, liste[start:end], 'hosp', 'Reg.76')

In [ ]:
start = end
end = start + 6
plot_x_y(df_R76, liste[start:end], 'hosp', 'Reg.76')

In [ ]:
start = end
end = start + 7
plot_x_y(df_R76, liste[start:end], 'hosp', 'Reg.76')

In [ ]:
# METHODOLOGY - Data understanding - Correlation with targets
# ----------------------------------------------------------------------========
# plot labels for target 'hosp' at level of Department Haute-Garonne
start = 0
end = start + 6
plot_x_y(df_D31, liste[start:end], 'hosp', 'Dep.31')

In [ ]:
start = end
end = start + 6
plot_x_y(df_D31, liste[start:end], 'hosp', 'Dep.31')

In [ ]:
start = end
end = start + 6
plot_x_y(df_D31, liste[start:end], 'hosp', 'Dep.31')

In [ ]:
start = end
end = start + 6
plot_x_y(df_D31, liste[start:end], 'hosp', 'Dep.31')

In [ ]:
start = end
end = start + 7
plot_x_y(df_D31, liste[start:end], 'hosp', 'Dep.31')

In [ ]:
# METHODOLOGY - Data understanding - Correlation with targets
# ----------------------------------------------------------------------========
# plot labels for target 'rea' at level of France

# Remove label 'rea' from the list to plot
start = 0
end = start + 6
plot_x_y(df_Fr, liste[start:end], 'rea', 'France')

In [ ]:
start = end
end = start + 6
plot_x_y(df_Fr, liste[start:end], 'rea', 'France')

In [ ]:
start = end
end = start + 6
plot_x_y(df_Fr, liste[start:end], 'rea', 'France')

In [ ]:
start = end
end = start + 6
plot_x_y(df_Fr, liste[start:end], 'rea', 'France')

In [ ]:
start = end
end = start + 7
plot_x_y(df_Fr, liste[start:end], 'rea', 'France')

In [ ]:
# METHODOLOGY - Data understanding - Correlation with targets
# ----------------------------------------------------------------------========
# plot labels for target 'rea' at level of Occitanie
start = 0
end = start + 6
plot_x_y(df_R76, liste[start:end], 'rea', 'Reg.76')

In [ ]:
start = end
end = start + 6
plot_x_y(df_R76, liste[start:end], 'rea', 'Reg.76')

In [ ]:
start = end
end = start + 6
plot_x_y(df_R76, liste[start:end], 'rea', 'Reg.76')

In [ ]:
start = end
end = start + 6
plot_x_y(df_R76, liste[start:end], 'rea', 'Reg.76')

In [ ]:
start = end
end = start + 7
plot_x_y(df_R76, liste[start:end], 'rea', 'Reg.76')

In [ ]:
# METHODOLOGY - Data understanding - Correlation with targets
# ----------------------------------------------------------------------========
# plot labels for target 'rea' at level of Haute-Garonne
start = 0
end = start + 6
plot_x_y(df_D31, liste[start:end], 'rea', 'Dep.31')

In [ ]:
start = end
end = start + 6
plot_x_y(df_D31, liste[start:end], 'rea', 'Dep.31')

In [ ]:
start = end
end = start + 6
plot_x_y(df_D31, liste[start:end], 'rea', 'Dep.31')

In [ ]:
start = end
end = start + 6
plot_x_y(df_D31, liste[start:end], 'rea', 'Dep.31')

In [ ]:
start = end
end = start + 7
plot_x_y(df_D31, liste[start:end], 'rea', 'Dep.31')

In [ ]:
# METHODOLOGY - Data understanding - Assumptions
# ----------------------------------------------------------------------========
# Need to plot time series
# Liste of departments in Region Occitanie
d_list = [9, 11, 12, 30, 31, 32, 34, 46, 48, 65, 66, 81, 82]

# Group the data set by dep and by timestamp to ease searching results
#  by department and by time.
if 'dep' in df.columns:
    df.groupby(['dep', 'timestamp'])  # If no dummy applied
else:
    df.groupby(['timestamp'])  # If dummy applied

# Plot time series for departments of the Region
"""
plot_time_series(df_, label='TO', departments=d_list)
plot_time_series(df_, label='rea', departments=d_list)
plot_time_series(df_, label='tx_pos', departments=d_list)
plot_time_series(df_, label='tx_incid', departments=d_list)
plot_time_series(df_, label='rad', departments=d_list)
plot_time_series(df_, label='dchosp', departments=d_list)
plot_time_series(df_, label='incid_hosp', departments=d_list)
plot_time_series(df_, label='incid_rea', departments=d_list)
plot_time_series(df_, label='incid_dchosp', departments=d_list)
plot_time_series(df_, label='pos', departments=d_list)
plot_time_series(df_, label='pos_7j', departments=d_list)
plot_time_series(df_, label='n_dose1', departments=d_list)
plot_time_series(df_, label='n_2_rappel', departments=d_list)
plot_time_series(df_, label='n_rappel_biv', departments=d_list)
plot_time_series(df_, label='n_3_rappel', departments=d_list)
plot_time_series(df_, label='n_cum_dose1', departments=d_list)
plot_time_series(df_, label='n_cum_rappel', departments=d_list)
plot_time_series(df_, label='n_cum_2_rappel', departments=d_list)
plot_time_series(df_, label='n_cum_rappel_biv', departments=d_list)
plot_time_series(df_, label='n_cum_3_rappel', departments=d_list)
plot_time_series(df_, label='couv_dose1', departments=d_list)
plot_time_series(df_, label='couv_complet', departments=d_list)
plot_time_series(df_, label='couv_rappel', departments=d_list)
plot_time_series(df_, label='couv_2_rappel', departments=d_list)
plot_time_series(df_, label='couv_rappel_biv', departments=d_list)
plot_time_series(df_, label='couv_3_rappel', departments=d_list)
plot_time_series(df_, label='n_cum_rappel_biv', departments=d_list)
"""
# Instruction: activate the line according to your needs
plot_time_series(df_, label='n_complet', departments=d_list)
plot_time_series(df_, label='n_cum_complet', departments=d_list)
plot_time_series(df_, label='n_rappel', departments=d_list)
plot_time_series(df_, label='hosp', departments=d_list)


In [ ]:
# METHODOLOGY - Main Data extraction
# ----------------------------------------------------------------------========
# prepare dataframes to be able to train and test on a target without the other target
df_Fr_hosp = df_Fr.drop(columns=['rea'])
df_Fr_rea = df_Fr.drop(columns=['hosp'])
df_R76_hosp = df_R76.drop(columns=['rea'])
df_R76_rea = df_R76.drop(columns=['hosp'])
df_D31_hosp = df_D31.drop(columns=['rea'])
df_D31_rea = df_D31.drop(columns=['hosp'])

In [ ]:
# METHODOLOGY - Implementation - modeling preparation
# ----------------------------------------------------------------------========
# Create a function for spliting the dataset into explanatory and
#  response data sets.

def get_X_y(df: object, response:str):
    '''
    DESCRIPTION
        Split df into exploratory X data and response y data
    INPUT
           df : pandas dataframe
           response : target variable
    OUTPUT
           X : A matrix holding all of the variables you want to
               consider when predicting the response
           y : the corresponding response vector
    '''
    # output
    X, y = None, None

    # Get the Response var
    if response in df.columns:
        # Split into explanatory and response variables (1/2)
        #  Get response variable
        y = df[response]
        df = df.drop(columns=[response])  # Remove pred_name from df
    else:
        print("\tCAUTION! Unable to find the response in df")
        y = None

    # Get the Exploratory vars
    # Split into explanatory and response variables (2/2)
    #  Get the input variables i.e. at this level just a copy of df
    X = df.copy(deep=True)


    if X is None:
        print('\tCAUTION! No X found')
    if y is None:
        print(' - No y found')

    del df, response
    return X, y

In [ ]:
# METHODOLOGY - Implementation - modeling preparation
# ----------------------------------------------------------------------========
# Create a function for getting a trained model with possibility to
#  select various the model

def get_model(model_name, X: object, y: object, testrate=.3,
              alpha_value=.1, n_neighbors_value=5):
    '''
    DESCRIPTION
        Returns a linear prediction model according to train data
    INPUT
           model_name : name of the selected model to use.
                        possible choice: 'LinearRegression',
                        'KNeighborsClassifier', 'LogisticRegression',
                        'Ridge', 'SVC.linear', 'SVC.poly', 'SVC.rbf',
                        'DecisionTreeClassifier', 'OneVsRestClassifier',
                        'GradientBoostingClassifier',
                        'RandomForestClassifier',
                        'GradientBoostingRegressor',
                        'GradientBoostingClassifier',
                        'NeighborsRegressor_uniform',
                        'NeighborsRegressor_distance',
                        'Lasso'
           X          : explanatory variables object
           y          : response variable object
           testrate   : proportion of the dataset to include in
                         the test split,between 0.0 and 1.0;
                         default value = 0.3
    OUTPUT
            model : linear regression model object from sklearn
            list of X_train and y_train
            list of X_test and y_test
    '''
    model = None
    Xtrain, Xtest, ytrain, ytest = None, None, None, None

    # sub-variables
    y_pred, split = None, False
    
    if (X is not None) and (y is not None):
        # Split into train and test X/y data set
        #  to establish the model and score it
        # print("- Training")
        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
                                        test_size=testrate,
                                        random_state=42)
        if (Xtrain is not None) and (Xtest is not None) and \
           (ytrain is not None) and (ytest is not None):
            split = True

        # Establish model
        # print("- Modeling")
        if split:

            # Linear model from scikit-learn:
            # https://scikit-learn.org/stable/tutorial/
            #  statistical_inference/supervised_learning.html

            # LINEAR REGRESSION MODELS
            # https://scikit-learn.org/stable/modules/linear_model.html#
            if model_name == 'LinearRegression':
                model = LinearRegression()

            # Ridge regression and classification
            if model_name == 'Ridge':
                model = Ridge(alpha=alpha_value)

            if model_name == 'LogisticRegression':
                model = LogisticRegression(C=50, max_iter=200)  # C=1e5
                # C parameter controls the amount of regularization in
                #  the LogisticRegression object:
                #  a large value for C results in less regularization.
                #  penalty="l2" gives Shrinkage (i.e. non-sparse
                #  coefficients), while penalty="l1" gives Sparsity.

            if model_name == 'Lasso':
                model = linear_model.Lasso(alpha=0.1)


            # SUPPORT VECTOR MACHINES
            # A priori, not weel adapted for the amount of data of my
            #  data set ; could use LinearSVC or SGDClassifier
            if model_name == 'SVC.linear':
                model = SVC(kernel='linear')

            if model_name == 'SVC.poly':
                model = SVC(kernel='poly', degree=3)

            if model_name == 'SVC.rbf':
                model = SVC(kernel='rbf')


            # NEAREST NEIGHBORS
            #  It would work well on my continuous data
            #   but less on 'last-24h' data.
            if model_name == 'NeighborsRegressor_uniform':
                model = neighbors.KNeighborsRegressor(weights='uniform',
                            n_neighbors = n_neighbors_value)

            if model_name == 'NeighborsRegressor_distance':
                model =neighbors.KNeighborsRegressor(weights='distance')

            if model_name == 'KNeighborsClassifier':
                model = KNeighborsClassifier()  # n_neighbors=5


            # DECISION TREE
            # src: https://scikit-learn.org/stable/modules/multiclass.html
            if model_name == 'DecisionTreeClassifier':
                model = DecisionTreeClassifier()  # random_state=0
                # source: https://scikit-learn.org/stable/modules/
                #  generated/sklearn.tree.DecisionTreeClassifier.html#
                #  sklearn.tree.DecisionTreeClassifier

            # ENSEMBLE METHODS
            if model_name == 'GradientBoostingRegressor':
                model = GradientBoostingRegressor()  # random_state=0            
                # source: https://scikit-learn.org/stable/modules/
                #  generated/sklearn.ensemble.GradientBoostingRegressor
                #  .html

            if model_name == 'GradientBoostingClassifier':
                model = GradientBoostingClassifier()
                # n_estimators=100, learning_rate=1.0, max_depth=1,
                #  random_state=0

            if model_name == 'RandomForestClassifier':
                model = RandomForestClassifier()
                # source: https://scikit-learn.org/stable/modules/
                #  generated/sklearn.ensemble.RandomForestClassifier
                #  .html#sklearn.ensemble.RandomForestClassifier
 
 
            # fit the model
            model.fit(Xtrain, ytrain)

            # Test the model on test data
            y_pred = model.predict(Xtest)

        else:
            y_pred = None
            print("\tCAUTION! No Model defined")
 
    return model, [Xtrain, ytrain], [Xtest, ytest]

In [ ]:
# METHODOLOGY - Implementation - modeling preparation
# ----------------------------------------------------------------------========
# Create a function for evualuating performance of the model

def get_model_performance(model, Xy_train, Xy_test, Xy):
    '''
    DESCRIPTION
        Evaluate the model by computing its R2 score.
    INPUT
        model is the model to evaluate
        Xy_train is a list of train data: [X_train, y_train]
        Xy_test is a list of test data: [X_test, y_test]
        Xy is a list of with all data: [X, y]
    OUTPUT
        dictionary of scores with values
        # score (float) in the measure of the model's performance
    '''
    
    # Get data sets
    X_train, y_train = Xy_train
    X_test, y_test = Xy_test
    X_true, y_true = Xy

    # Evaluate this model by gettings metrics with a model by
    #  regression: Get the model's score

    # Accuracy_score (https://scikit-learn.org/stable/modules/
    # generated/sklearn.metrics.accuracy_score.html)
    # Confusion matrix (https://scikit-learn.org/stable/modules/
    # generated/sklearn.metrics.confusion_matrix.html)
    #  not appropriate for my purpose
    # Common pitfalls (https://scikit-learn.org/stable/common_
    #  pitfalls.html): mean_sqaured_error and r2_score

    # According to https://stackoverflow.com/questions/37367405/
    #  python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
    # ... Accuracy score is only for classification problems
    # acc_score_train = accuracy_score(y_train, model.predict(X_train))
    # acc_score_test = accuracy_score(y_test, model.predict(X_test))
    # if debug: print("    - acc_score_train:", acc_score_train)
    # if debug: print("    - acc_score_test:", acc_score_test)

    # always according to https://stackoverflow.com/questions/37367405/
    #  python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
    # For regression problems, use: R2 Score, MSE (Mean Squared Error),
    # RMSE (Root Mean Squared Error).
    


    # srce: https://scikit-learn.org/0.15/modules/model_evaluation.html
    # and https://scikit-learn.org/stable/modules/model_evaluation.html


    # Regression scoring
    try:
        r2_score_value = r2_score(y_true, model.predict(X_true))
    except:
        r2_score_value = None
    try:
        explained_variance_score = explained_variance_score(y_true, model.predict(X_true))
    except:
        explained_variance_score = None


    # Classification metrics: For multilabel case scoring
    try:
        accuracy_score_value = accuracy_score(y_true,
                                              model.predict(X_true))
    except:
        accuracy_score_value = None
    try:
        precision_score_value = precision_score(y_true,
                                                model.predict(X_true),
                                                average='micro') 
    except:
        precision_score_value = None
    try:
        top_k_value = top_k_accuracy_score(y_true, model.predict(X_true),
                                           k=2)
    except:
        top_k_value = None
    try:
        balanced_value = balanced_accuracy_score(y_true,
                                                 model.predict(X_true))
    except:
        balanced_value = None
    try:
        # source: https://scikit-learn.org/0.15/modules/generated/sklearn
        #  .metrics.f1_score.html#sklearn.metrics.f1_score
        f1_score_value = f1_score(y_true, model.predict(X_true),
                                  average='micro')
    except:
        f1_score_value = None

    try:
        recall_score_value = recall_score(y_true, model.predict(X_true),
                                          average='micro') 
    except:
        recall_score_value = None
    try:
        fscore_value = precision_recall_fscore_support(y_true,
                                                model.predict(X_true),
                                                average='micro')
        # srce: https://scikit-learn.org/0.15/modules/generated/sklearn
        # .metrics.precision_recall_fscore_support.html#sklearn.metrics
        # .precision_recall_fscore_support
    except:
        fscore_value = None        

    # try:
    #   jaccard = jaccard_similarity_score(y_true, model.predict(X_true))
    # except:
    #   jaccard = None

    # source: https://scikit-learn.org/stable/modules/generated/sklearn
    # .ensemble.GradientBoostingRegressor.html#sklearn.ensemble
    # .GradientBoostingRegressor.score
    # source: https://www.kaggle.com/discussions/getting-started/27261
    try:
        score_value = model.score(X_test, y_test)
    except:
        score_value = None

    # Cross-validation
    # source: https://scikit-learn.org/stable/modules/cross_validation.html
    try:
        cross_val = cross_val_score(model, X_true, y_true)
        cross_val_mean = cross_val.mean()
        cross_val_stddev = cross_val.std()
    except:
        cross_val_mean = None
        cross_val_stddev = None

    # 'jaccard': Not available
    # 'cross_val_stddev', 'cross_val_mean', 'recall_score'
    #  ... generates issues due to too low number of iterations.
    return {'r2_score': r2_score_value,
    'explained_variance_score': explained_variance_score,
     'accuracy_score': accuracy_score_value,
     'top_k': top_k_value,
     'balanced': balanced_value,
     'fscore': fscore_value,
     'f1_score': f1_score_value,
     'score': score_value,
     'precision_score': precision_score_value}

In [ ]:
# METHODOLOGY - Implementation - modeling preparation
# ----------------------------------------------------------------------========
# Create a function for indicating the weight of every parameters

def coef_weights(model, X) -> object:
    '''
    DESCRIPTION
       Returns a dataframe with coefficients of the model
        (real and absolute values) sorted in the descending order
        of the absolute values
    INPUT       
       model is the model for which we are looking coefficients
       X is the exploratory set of data
    OUTPUT
      coefs_df : model's coefficients that can be used to understand
                 the most influential coefficients in a linear model
                 by providing the coefficient estimates along with the
                 name of the variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    # Get name of every column in front  of its coefficients
    coefs_df['est_int'] = X.columns
    # get coefficients of the linear model
    coefs_df['coefs'] = model.coef_
    # get absolute value of these coefficients
    coefs_df['abs_coefs'] = np.abs(model.coef_)
    # Sort coefficient by descending order
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)

    return coefs_df

In [ ]:
# METHODOLOGY - Implementation - modeling preparation
# ----------------------------------------------------------------------========
# Create a function that allows launching the modeling

def process_model(df, response, model_name, drop_list=[], display=True,
                  alpha=.1, n_neighbors=5, scoring_display=[]):
    '''
    DESCRIPTION
        Define, evaluate and discuss a model
    INPUT
        df is the pandas dataframe with data to build and evaluate the 
           model;
        response (string) is the category / label of the column defined
                 as response/target for the model to define;
        model_name  is name of the selected model to use.
                     possible choice: 'LinearRegression',
                     'KNeighborsClassifier', 'LogisticRegression',
                     'Ridge', 'SVC.linear', 'SVC.poly', 'SVC.rbf',
                     'DecisionTreeClassifier', 'OneVsRestClassifier',
                     'GradientBoostingClassifier',
                     'RandomForestClassifier',
                     'GradientBoostingRegressor',
                     'GradientBoostingClassifier',
                     'NeighborsRegressor_uniform',
                     'NeighborsRegressor_distance',
                     'Lasso'
        drop_list (list) is a list of categories / labels of the columns
                  that can be removed from the dataframe before 
                  defining and evaluating the model. It does not modify
                  input dataframe. Default = nil (empty list)
        display  is .................
        alpha  is .................
        n_neighbors  is ................
        scoring_display  is a list of the scores to display;
                         if [] i.e. default, display all scores.
    OUTPUT
        model is the trained model define to predict the response
              according to the other labels of the dataframe.
        scores is a dictionaru of metrics to measure the performance of
               the model.coef_, closer to 1, better.
        coefs is the list of labels of the dataframe used to establish
              the model for which a weight (contribution to the model)
              is computed (signed and absolute values).
    '''
    model, scores, coefs = None, None, None

    df_mdl = df.copy(deep=True)
    if display:
        print('Modeling...')
    # Remove this list of labels
    if display:
        print('- drop')
    if len(drop_list) > 0:
        df_mdl.drop(columns=drop_list, inplace=True)

    # Split into Response y / Exploratory variables X
    if display:
        print("- Split into X/y")
    X, y = get_X_y(df_mdl, response)

    # modeling
    if display:
        print("- Get model")
    model, Xy_train, Xy_test = get_model(model_name, X, y,
                                alpha_value=alpha,
                                n_neighbors_value=n_neighbors)

    # Check performance
    if display:
        print("- Performance")
    scores = get_model_performance(model, Xy_train, Xy_test, [X, y])
    for score in scores:
        if (scoring_display == []) or (score in scoring_display):
            print("\tscore:", score, scores[score])
        
    # Coefficients of weight
    if display:
        print("- Model's coefficients")
        coefs = coef_weights(model, X)
        pd.set_option("display.max_rows", None,
                      "display.max_columns", None)    
        print(coefs)

    return model, scores, coefs

In [ ]:
# METHODOLOGY - Implementation - model selection
# ----------------------------------------------------------------------========

# Create a function that allows applying a list of models on a given 
#  dataframe to a given target label of the dataframe.
def apply_models(models_list, dff, response, alpha_value=0.1,
                 n_neighbors_value=5, scoring_sts=[]):
    """
    DESCRIPTION
        Apply a list of models on a given dataframe to a given target
         label of the dataframe.
    INPUT
        models_list  is a list of models' names (string) as defined in
                      the function 'get_model'.
        dff  is the dataframe to work on for training and assessing
              a model.
        response  is the target label (string) of the modeling.
        alpha_value  is ..............
        n_neighbors_value  is ..............
        scoring_sts  is .................
    OUTPUT
        none : display of scores for every tested model
    """
    for model_name in models_list:
        print('\nModel:', model_name)
        model, score, coefs = process_model(dff, response, model_name,
                                display=False,
                                alpha=alpha_value,
                                n_neighbors = n_neighbors_value,
                                scoring_display = scoring_sts)


# Define a list of models to test
models_list = ['LinearRegression', 'Ridge', 
               'NeighborsRegressor_distance']
# very long: 'SVC.linear', OneVsRestClassifier even on short data set
# Negative R2 score: 'GradientBoostingClassifier', 'SVC.poly', 'SVC.rbf'
# Do not converge: 'LogisticRegression'
# Generate warning: 'KNeighborsClassifier', 'GradientBoostingRegressor',
#                   'DecisionTreeClassifier', 'GradientBoostingClassifier',
#                   'RandomForestClassifier',
# need to increase the number of iterations: 'NeighborsRegressor_uniform',
#                                            'Lasso'

In [ ]:
# METHODOLOGY - Implementation - model selection
# ----------------------------------------------------------------------========
apply_models(models_list, df_D31_hosp, 'hosp')

In [ ]:
# METHODOLOGY - Implementation - model selection
# ----------------------------------------------------------------------========
apply_models(models_list, df_R76_hosp, 'hosp')

In [ ]:
# METHODOLOGY - Implementation - model selection
# ----------------------------------------------------------------------========
# apply_models(models_list, df_Fr_hosp, 'hosp')
# deactivated because too long (> 10min)

In [ ]:
# METHODOLOGY - Implementation - model selection
# ----------------------------------------------------------------------========
apply_models(models_list, df_D31_rea, 'rea')

In [ ]:
# METHODOLOGY - Implementation - model selection
# ----------------------------------------------------------------------========
apply_models(models_list, df_R76_rea, 'rea')

In [ ]:
# METHODOLOGY - Implementation - model selection
# ----------------------------------------------------------------------========
# apply_models(models_list, df_Fr_rea, 'rea')
# deactivated because too long (> 10min)

In [ ]:
# METHODOLOGY - Implementation - Refinement
# ----------------------------------------------------------------------========
# remove last-24h data
# df_D31_hosp.columns
# result = ['tx_pos', 'tx_incid', 'TO', 'hosp', 'rad', 'dchosp', 'reg_rea',
#       'incid_hosp', 'incid_rea', 'incid_rad', 'incid_dchosp', 'reg_incid_rea',
#       'pos', 'pos_7j', 'n_dose1', 'n_complet', 'n_rappel', 'n_2_rappel',
#       'n_rappel_biv', 'n_3_rappel', 'n_cum_dose1', 'n_cum_complet',
#       'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_rappel_biv', 'n_cum_3_rappel',
#       'couv_dose1', 'couv_complet', 'couv_rappel', 'couv_2_rappel',
#       'couv_rappel_biv', 'couv_3_rappel']
new_drop_list = ['tx_incid', 'incid_hosp', 'incid_rea', 'incid_rad',
                 'incid_dchosp', 'reg_incid_rea']

# remove these parameters from the dataframes (work on a copy)
df_D31_hosp_red = df_D31_hosp.copy(deep=True)
df_R76_hosp_red = df_R76_hosp.copy(deep=True)
df_Fr_hosp_red = df_Fr_hosp.copy(deep=True)
df_D31_hosp_red = df_D31_hosp_red.drop(columns=new_drop_list)
df_R76_hosp_red = df_R76_hosp_red.drop(columns=new_drop_list)
df_Fr_hosp_red = df_Fr_hosp_red.drop(columns=new_drop_list)

# check of hosp predictions
models_list = ['NeighborsRegressor_distance']
target = 'hosp'
print('DEPARTMENT:')
apply_models(models_list, df_D31_hosp_red, target)
print('REGION:')
apply_models(models_list, df_R76_hosp_red, target)
#print('FRANCE:')  # deactivated because too long (> 10min)
#apply_models(models_list, df_Fr_hosp_red, target)

In [ ]:
# check of hosp predictions
models_list = ['NeighborsRegressor_distance']
target = 'hosp'
n_neighbors = 2
print('DEPARTMENT:')
apply_models(models_list, df_D31_hosp_red, target, n_neighbors_value=n_neighbors)
print('REGION:')
apply_models(models_list, df_R76_hosp_red, target, n_neighbors_value=n_neighbors)
#print('FRANCE:')  # deactivated because too long (> 10min)
#apply_models(models_list, df_Fr_hosp_red, target, n_neighbors_value=n_neighbors)

In [ ]:
# check of hosp predictions
models_list = ['NeighborsRegressor_distance']
target = 'hosp'
n_neighbors = 15
print('DEPARTMENT:')
apply_models(models_list, df_D31_hosp_red, target, n_neighbors_value=n_neighbors)
print('REGION:')
apply_models(models_list, df_R76_hosp_red, target, n_neighbors_value=n_neighbors)
#print('FRANCE:')  # deactivated because too long (> 10min)
#apply_models(models_list, df_Fr_hosp_red, target, n_neighbors_value=n_neighbors)

In [ ]:
# METHODOLOGY - Implementation - Refinement
# ----------------------------------------------------------------------========
# Test change of alpha on Ridge modeling method: from 0.1 (default)
#  to 0.5
alpha = 0.5

# check of hosp predictions
models_list = ['Ridge']
target = 'hosp'
print('DEPARTMENT:')
apply_models(models_list, df_D31_hosp, target, alpha_value=alpha)
print('REGION:')
apply_models(models_list, df_R76_hosp, target, alpha_value=alpha)
print('FRANCE:')
apply_models(models_list, df_Fr_hosp, target, alpha_value=alpha)

In [ ]:
"""# METHODOLOGY - Implementation - Refinement
# ----------------------------------------------------------------------========
# Remove the last-24h data from the data sets
new_drop_list = ['tx_incid', 'incid_hosp', 'incid_rea', 'incid_rad',
                 'incid_dchosp', 'reg_incid_rea']
df2 = df.drop(columns=new_drop_list)
df_D31_hosp2 = df_D31_hosp.drop(columns=new_drop_list)
df_R76_hosp2 = df_R76_hosp.drop(columns=new_drop_list)
df_Fr_hosp2 = df_Fr_hosp.drop(columns=new_drop_list)
df_D31_rea2 = df_D31_rea.drop(columns=new_drop_list)
df_R76_rea2 = df_R76_rea.drop(columns=new_drop_list)
df_Fr_rea2 = df_Fr_rea.drop(columns=new_drop_list)
"""

In [ ]:
def applying_model(df, model_name=None, response='', testrate=.3):

    df_mdl = df.copy(deep=True)

    # Define data
    X, y = get_X_y(df_mdl, response)

    # split data 
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=testrate,
                                    random_state=42)
    # Modeling
    model = None
    if model_name == 'LinearRegression':
        model = LinearRegression()
    if model_name == 'Ridge':
        model = Ridge()
    if model_name == 'NeighborsRegressor_distance':
        model =neighbors.KNeighborsRegressor(weights='distance')

    # fit and apply
    if model is not None:
        model.fit(X_train, y_train)  # fit the model
        y_pred = model.predict(X_test)  # Test the model on test data

    # Scoring
    try:
        r2_score_value = r2_score(y, model.predict(X))
    except:
        r2_score_value = None
    try:
        score_value = model.score(X_test, y_test)
    except:
        score_value = None

    return {'r2_score': r2_score_value, 'score': score_value}

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# I would try to reduce the number of parameters by identifying once
#  that provide significant result and other that do not.

def get_overall_scores(df, target):
    """
    DESCRIPTION
        get score of selected model for every parameter; store result
         into a dictionary. 
    INPUT
        dfo  is the dataframe
        target  is the name of the parameter used as answer (str)
    OUTPUT
        overall_scores  is a dictionary of model method, providing
                        for every method a dictionary of input
                        parameters; gives scores for all methods and
                        all param:
                   {methode1: {p1: {'r2_score': value, 'score': value},
                               p2: {}, ...},
                    methode2: {...} 
                   }
    """
    overall_scores = dict()

    models_list = ['LinearRegression', 'Ridge', 
                'NeighborsRegressor_distance']

    # get the list of input parameters:
    dfo = df.copy(deep=True)
    ip_list = dfo.drop(columns=[target]).columns

    # for every, modeling method, get scores while using every input 
    # parameter:
    for model in models_list:
        overall_scores[model] = dict()    
        for ip in ip_list:
            df_short =dfo[[target, ip]]
            # print('\n>> Model', model, 'param:', ip)
            scores = applying_model(df_short, model_name=model, response=target)
            overall_scores[model][ip] = scores

    del dfo, target, ip_list, models_list
    return overall_scores

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# In first approach, to quickly get result, focus on the department.
overall_scores_hosp_D31 = get_overall_scores(df_D31_hosp, 'hosp')

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# Then, I will sort score related to all parameters for every method,
# in order to display onces bringing highest scores to lowest.

def display_overall_scores(overall_scores, target, score_name):
    """
    DESCRIPTION
        Display a plot of score per input parameter and per model.
    INPUT
        overall_scores  is a dictionary of model method, providing
                        for every method a dictionary of input
                        parameters; gives scores for all methods and
                        all param:
                   {methode1: {p1: {'r2_score': value, 'score': value},
                               p2: {}, ...},
                    methode2: {...} 
                   }
        target  is the name of the parameter selected as answer.
        score_name  is the name of the selected type of score.
                    choose among variable 'scoring_list'
    OUTPUT
        none (display)
    """

    models_list = ['LinearRegression', 'Ridge', 
                   'NeighborsRegressor_distance']

    dfp = pd.DataFrame([])
    params, scores, models = [], [], []
    for i, model in enumerate (models_list):
        for param in overall_scores[model]:
            params.append(param)
            scores.append(overall_scores[model][param][score_name])
            models.append(model)

    dfp['parameters'] = params
    dfp['scores'] = scores
    dfp['model'] = models
    
    fig = px.bar(dfp, x="parameters", y="scores",
                color="model",
                title="Scores with three modeling methods (default settings) for response '" + target + "'",
                barmode = 'group')
    fig.show()

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# display the result of 'hosp' and at level of department
display_overall_scores(overall_scores_hosp_D31, 'hosp', 'score')

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# This figure show that some input parameters has a significant impact
#  on the scores whatever the modeling method, and some others depend on
#  the modeling method.
# Such that I assume I can remove somme non significant parameters,
#  i.e parameters that give a relative low positive score (not really
#  visible on the plot) or a negative score:
LinearRegression_keepinglist_hosp = ['tx_pos', 'tx_incid', 'TO',
 'reg_rea', 'incid_hosp', 'incid_rea', 'incid_rad', 'incid_dchosp',
 'reg_incid_rea', 'pos', 'pos_7j', 'n_dose1', 'n_complet',
 'n_cum_rappel', 'couv_rappel']
LinearRidge_keepinglist_hosp = LinearRegression_keepinglist_hosp
Neighbors_Regressor_keepinglist_hosp = ['tx_pos', 'tx_incid', 'rad',
 'dchosp', 'incid_hosp', 'incid_rad', 'reg_incid_rea', 'pos', 'pos_7j',
 'n_cum_dose1', 'n_cum_complet', 'n_cum_rappel', 'n_cum_2_rappel',
 'n_cum_3_rappel', 'couv_dose1', 'couv_complet', 'couv_rappel',
 'couv_2_rappel']

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# same work for the second target with a focus on the department.
target = 'rea'
overall_scores_rea_D31 = get_overall_scores(df_D31_rea, target)
display_overall_scores(overall_scores_rea_D31, target, 'score')

In [ ]:
 # METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# This figure show that some input parameters has a significant impact
#  on the scores whatever the modeling method, and some others depend on
#  the modeling method.
# Such that I assume I can remove somme non significant parameters:
LinearRegression_keepinglist_rea = ['tx_incid', 'TO', 'rad', 'dchosp',
 'reg_rea', 'incid_hosp', 'incid_rea', 'incid_rad', 'incid_dchosp',
 'reg_incid_rea', 'pos', 'pos_7j', 'n_rappel', 'n_2_rappel',
 'n_cum_dose1', 'n_cum_complet', 'n_cum_rappel', 'n_cum_2_rappel',
 'n_cum_3_rappel', 'couv_dose1', 'couv_complet', 'couv_rappel',
 'couv_2_rappel', 'couv_rappel_biv', 'couv_3_rappel']
LinearRidge_keepinglist_rea = LinearRegression_keepinglist_rea
Neighbors_Regressor_keepinglist_rea = ['tx_pos', 'tx_incid', 'TO',
 'rad', 'dchosp', 'reg_rea', 'incid_hosp', 'reg_incid_rea', 'pos_7j',
  'n_dose1', 'n_complet', 'n_rappel', 'n_cum_dose1', 'n_cum_complet',
  'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_3_rappel', 'couv_dose1',
  'couv_complet', 'couv_rappel', 'couv_2_rappel']

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# Create new data sets only with parameters to keep as identified
#  here-above for the three modeling methods. (always at department
#  level).
def compare_df_columns(df1, df2, title=''):
    """
    DESCRIPTION
        Display difference of columns' name between two dataframes.
    INPUT
        df1  is the first dataframe to compare
        df2  is the second dataframe to compare
    OUTPUT
        none (only display)
    """
    l1 = df1.columns
    l2 = df2.columns

    # Work as if df2 as lower columns than df1.
    # Indicate how many stay and how many leave
    nb_stay, nb_leave = 0, 0
    # Compute nb kept
    for n2 in l2:
        for n1 in l1:   
            if n2 == n1:
                nb_stay += 1
    # Compute nb removed
    for n1 in l1:
        find = False
        for n2 in l2:
            if n1 == n2:
                find = True
        if not find:
            nb_leave += 1
    print(title + ':')
    print("\t{0} parameters kept and {1} parameters removed".format(nb_stay, nb_leave))


def keep_one_target(liste, target):
    """ Allows removing multiple occurrence of target when laucnhed
        several times"""
    new_list = []
    for elt in liste:
        if elt != target:
            new_list.append(elt)
    new_list.append(target)
    return new_list


target = 'hosp'

LinearRegression_keepinglist_hosp = keep_one_target(
    LinearRegression_keepinglist_hosp, target)
df_D31_hosp_LinReg = df_D31_hosp[LinearRegression_keepinglist_hosp]
compare_df_columns(df_D31_hosp, df_D31_hosp_LinReg,
            title="LinearRegression on '" + target + "' with D31")

LinearRidge_keepinglist_hosp = keep_one_target(
    LinearRidge_keepinglist_hosp, target)
df_D31_hosp_LinRid = df_D31_hosp[LinearRidge_keepinglist_hosp]
compare_df_columns(df_D31_hosp, df_D31_hosp_LinRid,
            title="LinearRidge on '" + target + "' with D31")

Neighbors_Regressor_keepinglist_hosp = keep_one_target(
    Neighbors_Regressor_keepinglist_hosp, target)
df_D31_hosp_NeiReg = df_D31_hosp[Neighbors_Regressor_keepinglist_hosp]
compare_df_columns(df_D31_hosp, df_D31_hosp_NeiReg,
            title="NeighborsRegression on '" + target + "' with D31")

target = 'rea'

LinearRegression_keepinglist_rea = keep_one_target(
    LinearRegression_keepinglist_rea, target)
df_D31_rea_LinReg = df_D31_rea[LinearRegression_keepinglist_rea]
compare_df_columns(df_D31_rea, df_D31_rea_LinReg,
            title="LinearRegression on '" + target + "' with D31")

LinearRidge_keepinglist_rea = keep_one_target(
    LinearRidge_keepinglist_rea, target)
df_D31_rea_LinRid = df_D31_rea[LinearRidge_keepinglist_rea]
compare_df_columns(df_D31_rea, df_D31_rea_LinRid,
            title="LinearRidge on '" + target + "' with D31")

Neighbors_Regressor_keepinglist_rea = keep_one_target(
    Neighbors_Regressor_keepinglist_rea, target)
df_D31_rea_NeiReg = df_D31_rea[Neighbors_Regressor_keepinglist_rea]
compare_df_columns(df_D31_rea, df_D31_rea_NeiReg,
            title="NeighborsRegression on '" + target + "' with D31")

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# Create new data sets only with parameters to keep as identified
#  here-above for the three modeling methods.
# Built the refined dataframes
#  
#  At Region level

target = 'hosp'

LinearRegression_keepinglist_hosp = keep_one_target(
    LinearRegression_keepinglist_hosp, target)
df_R76_hosp_LinReg = df_R76_hosp[LinearRegression_keepinglist_hosp]
compare_df_columns(df_R76_hosp, df_R76_hosp_LinReg,
            title="LinearRegression on '" + target + "' with R76")

LinearRidge_keepinglist_hosp = keep_one_target(
    LinearRidge_keepinglist_hosp, target)
df_R76_hosp_LinRid = df_R76_hosp[LinearRidge_keepinglist_hosp]
compare_df_columns(df_R76_hosp, df_R76_hosp_LinRid,
            title="LinearRidge on '" + target + "' with R76")

Neighbors_Regressor_keepinglist_hosp = keep_one_target(
    Neighbors_Regressor_keepinglist_hosp, target)
df_R76_hosp_NeiReg = df_R76_hosp[LinearRidge_keepinglist_hosp]
compare_df_columns(df_R76_hosp, df_R76_hosp_NeiReg,
            title="NeighborRegressor on '" + target + "' with R76")

target = 'rea'

LinearRegression_keepinglist_rea= keep_one_target(
    LinearRegression_keepinglist_rea, target)
df_R76_rea_LinReg = df_R76_rea[LinearRegression_keepinglist_rea]
compare_df_columns(df_R76_rea, df_R76_rea_LinReg,
            title="LinearRegression on '" + target + "' with R76")

LinearRidge_keepinglist_rea = keep_one_target(
    LinearRidge_keepinglist_rea, target)
df_R76_rea_LinRid = df_R76_rea[LinearRidge_keepinglist_rea]
compare_df_columns(df_R76_rea, df_R76_rea_LinRid,
            title="LinearRidge on '" + target + "' with R76")

Neighbors_Regressor_keepinglist_rea = keep_one_target(
    Neighbors_Regressor_keepinglist_rea, target)
df_R76_rea_NeiReg = df_R76_rea[LinearRidge_keepinglist_rea]
compare_df_columns(df_R76_rea, df_R76_rea_NeiReg,
            title="NeighborRegressor on '" + target + "' with R76")


# At France level

target = 'hosp'

LinearRegression_keepinglist_hosp = keep_one_target(
    LinearRegression_keepinglist_hosp, target)
df_Fr_hosp_LinReg = df_Fr_hosp[LinearRegression_keepinglist_hosp]
compare_df_columns(df_Fr_hosp, df_Fr_hosp_LinReg,
            title="LinearRegression on '" + target + "' with Fr")

LinearRidge_keepinglist_hosp = keep_one_target(
    LinearRidge_keepinglist_hosp, target)
df_Fr_hosp_LinRid = df_Fr_hosp[LinearRidge_keepinglist_hosp]
compare_df_columns(df_Fr_hosp, df_Fr_hosp_LinRid,
            title="LinearRidge on '" + target + "' with Fr")

Neighbors_Regressor_keepinglist_hosp = keep_one_target(
    Neighbors_Regressor_keepinglist_hosp, target)
df_Fr_hosp_NeiReg = df_Fr_hosp[LinearRidge_keepinglist_hosp]
compare_df_columns(df_Fr_hosp, df_Fr_hosp_NeiReg,
            title="NeighborRegressor on '" + target + "' with Fr")

target = 'rea'

LinearRegression_keepinglist_rea= keep_one_target(
    LinearRegression_keepinglist_rea, target)
df_Fr_rea_LinReg = df_Fr_rea[LinearRegression_keepinglist_rea]
compare_df_columns(df_Fr_rea, df_Fr_rea_LinReg,
            title="LinearRegression on '" + target + "' with Fr")

LinearRidge_keepinglist_rea = keep_one_target(
    LinearRidge_keepinglist_rea, target)
df_Fr_rea_LinRid = df_Fr_rea[LinearRidge_keepinglist_rea]
compare_df_columns(df_Fr_rea, df_Fr_rea_LinRid,
            title="LinearRidge on '" + target + "' with Fr")

Neighbors_Regressor_keepinglist_rea = keep_one_target(
    Neighbors_Regressor_keepinglist_rea, target)
df_Fr_rea_NeiReg = df_Fr_rea[LinearRidge_keepinglist_rea]
compare_df_columns(df_Fr_rea, df_Fr_rea_NeiReg,
            title="NeighborRegressor on '" + target + "' with Fr")

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# Create a function that will allows checking the impact of this change 
#  of data sets.

def get_scores(df, target, sel_model=None):
    """
    DESCRIPTION
        get scores for a selected dataframe and target
    INPUT
        dfr  is the dataframe
        target  is the parameter selected as answer
    OUTPUT
        overall_scores  is a dictionary of the dictionary's scores for 
                        all selected modeling methods.
                   {methode1: {'r2_score': value, 'score': value},
                    methode2: {...},
                    ...
                   }
    """
    overall_scores = dict()

    models_list = ['LinearRegression', 'Ridge', 
                'NeighborsRegressor_distance']

    dfr = df.copy(deep=True)

    # for every, modeling method, get scores while using every input 
    # parameter:
    for model in models_list:
        if (sel_model is None) or (model == sel_model):
            scores = applying_model(dfr, model_name=model, response=target)
            overall_scores[model] = scores

    del dfr, target
    return overall_scores

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# Create a function to display results, including comparison of scores
#  before and after reducing the parameters list (refinement).

def display_refinement_result(df_list, target):
    """
    DESCRIPTION
        Display result of the refinement process (reducing the number
         of paramters per data set) by comparison before and after
         reduction, for every model.
    INPUT
        df_list  is a list of two lists
                 (one before reduction, one after reduction)
                 ordered in the same order that the modeling methods.
        target  is the parameter selected as response (str).
    OUTPUT
        none (only display and plot)
    """
    models_list = ['LinearRegression', 'Ridge', 
                'NeighborsRegressor_distance']

    # Get the scores for every model on its dataframe
    scores_per_model = dict()
    for i, model in enumerate (models_list):
        scores = []
        for df_sublist in df_list: # before, then after
            scores_dict = get_scores(df_sublist[i],
                                    target, sel_model=model)
            scores.append(scores_dict[model]['score'])  # get only this type of score
        scores_per_model[model] = scores

    # display result
    before_list, after_list = [], []
    df_result = pd.DataFrame([])
    model_list, scores_list, status_list = [], [], []
    print('RESULTS:\t\t\t\tBefore refinement\t After refinement')
    for model in scores_per_model:
        print("Model '", model, "':", (27-len(model))*' ',
            scores_per_model[model][0], '\t', scores_per_model[model][1])
        # Manage df for further display
        for j, sts in enumerate (scores_per_model[model]):
            model_list.append(model)
            scores_list.append(scores_per_model[model][j])
            if j == 0:
                status = 'before'
                before_list.append(scores_per_model[model][j])
            else:
                status = 'after'
                after_list.append(scores_per_model[model][j])
            status_list.append(status)

    df_result['model'] = model_list
    df_result['score'] = scores_list
    df_result['status'] = status_list

    # Plot
    """
    # Not the most appropriate type of plot here.
    fig = px.bar(df_result, x="status", y="score",
                color="model",
                title="Comparison of scores with three modeling methods (default settings) for response '" + target + "'",
                barmode = 'group')
    fig.show()
    """

    x = models_list
    plot = go.Figure(data=[go.Bar(
        name = 'Before',
        x = x,
        y = before_list
    ),
                        go.Bar(
        name = 'After',
        x = x,
        y = after_list
    )
    ])
    #,
    #layout=go.Layout(
    #    title=go.layout.Title(text="Comparison of scores with three modeling methods (default settings) for response '" + target + "'")
    #))

    plot.update_layout(
        title="Comparison of scores with three modeling methods (default settings) for response '" + target + "'",
        xaxis_title="Modeling methods",
        yaxis_title="Scores",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    plot.show()

In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# Compare the scores before/after reducing the number of parameters in
#  the data set.

target = 'hosp'
df_list_before = [df_D31_hosp, df_D31_hosp, df_D31_hosp]
df_list_after = [df_D31_hosp_LinReg, df_D31_hosp_LinRid,\
                 df_D31_hosp_NeiReg]
df_list = [df_list_before, df_list_after]
# display result for 'hosp'
display_refinement_result(df_list, target)



In [ ]:
# METHODOLOGY - Data understanding - Refinement
# ----------------------------------------------------------------------========
# Compare the scores before/after reducing the number of parameters in
#  the data set.

target = 'rea'
df_list_before = [df_D31_rea, df_D31_rea, df_D31_rea]
df_list_after = [df_D31_rea_LinReg, df_D31_rea_LinRid,\
                 df_D31_rea_NeiReg]
df_list = [df_list_before, df_list_after]
# display result for 'hosp'
display_refinement_result(df_list, target)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Define a function to identify the best hyper parameter tuning over the
#   various modeling methods, targets and areas.

def search_hyperparameters_tuning(df, sel_mode='', sel_target=''):
    """
    DESCRIPTION
        Provide information to select the best hyper-parameter tuning 
         on the selection (dataframe, model and target).
    INPUT
        df  is the dataframe to work on
        sel_mode  is the name of the selected modeling method:
                  models_list = ['LinearRegression', 'Ridge', 
                  'NeighborsRegressor_distance']
        sel_target  is the name of the parameters used as answer.
    OUTPUT
        dictionary of arguments related to Random Search method:
        Available keys: 'best_param', 'scorer'
    """

    # source: https://scikit-learn.org/stable/modules/grid_search.html
    # and https://scikit-learn.org/stable/auto_examples/model_selection/
    #      plot_successive_halving_iterations.html#sphx-glr-auto-examples-
    #      model-selection-plot-successive-halving-iterations-py

    dfm = df.copy(deep=True)
    model = None
    parameters = []

    X, y = get_X_y(dfm, sel_target)
    # X, y = get_X_y(df_R76_hosp, 'hosp')
    # X, y = get_X_y(df_Fr_hosp, 'hosp')
    # X, y = get_X_y(df_D31_rea, 'rea')
    # X, y = get_X_y(df_R76_rea, 'rea')
    # X, y = get_X_y(df_Fr_rea, 'rea')

    # Model #1
    if sel_mode == 'LinearRegression':
        model = LinearRegression()
        parameters = {'fit_intercept': [True, False],
                    'positive': [True, False]}
    # with reduced data set:
    # RESULT hosp D31 best_params_: {'positive': True, 'fit_intercept': False}
    # RESULT hosp R76 best_params_: {'positive': True, 'fit_intercept': False}
    # RESULT hosp Fr  best_params_: {'positive': True, 'fit_intercept': False}
    # RESULT rea D31  best_params_: {'positive': True, 'fit_intercept': False}
    # RESULT rea R76  best_params_: {'positive': True, 'fit_intercept': False}
    # ...
    # with not reduced data set:
    # {'positive': True, 'fit_intercept': False}
    # {'positive': True, 'fit_intercept': False}
    # {'positive': True, 'fit_intercept': False}
    # {'positive': True, 'fit_intercept': False}
    # {'positive': True, 'fit_intercept': False}
    # {'positive': True, 'fit_intercept': False}

    # Model #2
    if sel_mode == 'Ridge':
        model = Ridge()  # fit_intercept=True
        parameters = {'fit_intercept': [True, False],
                    'alpha': [0.1, 0.5, 1, 5, 10, 20],
                    'positive': [True, False],
                    'max_iter': [10, 25, 50, 75, 100, 500, 600, 700, 800, 900, 1000],
                    'tol': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e-0, 10, 100]}
    # with reduced data set:
    # RESULT hosp D31 best_params: {'tol': 1e-05, 'positive': True, 'max_iter': 500, 'fit_intercept': True, 'alpha': 1}
    #                              {'tol': 1e-05, 'positive': True, 'max_iter': 500,                        'alpha': 1}
    # RESULT hosp R76 best_params: {'tol': 1e-05, 'positive': True, 'max_iter': 600, 'fit_intercept': True, 'alpha': 0.5}
    #                              {'tol': 1e-05, 'positive': True, 'max_iter': 25,                         'alpha': 1}
    # RESULT hosp Fr best_params_: {'tol': 1e-05, 'positive': True, 'max_iter': 500, 'fit_intercept': False, 'alpha': 1}
    # RESULT rea D31  best_params: {'tol': 0.0001, 'positive': True, 'max_iter': 75, 'fit_intercept': False, 'alpha': 20}
    # RESULT rea R76  best_params: {'tol': 0.1, 'positive': True, 'max_iter': 900, 'fit_intercept': False, 'alpha': 10}
    # RESULT rea Fr   best_params: {'tol': 1e-05, 'positive': True, 'max_iter': 900, 'fit_intercept': False, 'alpha': 1}

    # with not reduced data set:
    # RESULT hosp D31 best_params: {'tol': 1e-05, 'positive': True, 'max_iter': 100, 'fit_intercept': True, 'alpha': 1}
    # RESULT hosp R76 best_params: {'tol': 1e-05, 'positive': True, 'max_iter': 500, 'fit_intercept': True, 'alpha': 1}
    # RESULT hosp Fr best_params_: {'tol': 0.0001, 'positive': True, 'max_iter': 900, 'fit_intercept': False, 'alpha': 1}
    # RESULT hosp Fr best_params_: {'tol': 0.0001, 'positive': True, 'max_iter': 500,                  True   'alpha': 1}
    # RESULT rea D31  best_params: {'tol': 1.0, 'positive': True, 'max_iter': 10, 'fit_intercept': True, 'alpha': 0.1}
    # RESULT rea R76  best_params: {'tol': 1e-05, 'positive': True, 'max_iter': 25, 'fit_intercept': False, 'alpha': 0.5}
    # RESULT rea R76  best_params: {'tol': 1e-05, 'positive': True, 'max_iter': 600,                 True   'alpha': 0.1}
    # RESULT rea Fr   best_params: {'tol': 0.0001, 'positive': True, 'max_iter': 500, 'fit_intercept': True, 'alpha': 1}


    # Model #3
    if sel_mode == 'NeighborsRegressor_distance':
        model = neighbors.KNeighborsRegressor(weights='distance')
        parameters = {'n_neighbors': [1, 2, 4, 6, 8, 10, 20, 50, 100, 500, 600, 700, 800, 900, 1000],
                    'p': [1, 2],
                    'n_jobs': [1, 2, 3, 4, 5, 6, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
                    }
        #(model.fit(X, y)).get_params().keys()

    # with reduced data set:
    # RESULT hosp D31 best_params_: {'p': 1, 'n_neighbors': 500, 'n_jobs': 4}
    # RESULT hosp R76 best_params_: {'p': 1, 'n_neighbors': 900, 'n_jobs': 4}
    # RESULT hosp Fr  best_params_: {'p': 1, 'n_neighbors': 900, 'n_jobs': 4}
    # RESULT rea D31  best_params_: {'p': 1, 'n_neighbors': 800, 'n_jobs': 50}
    # RESULT rea R76  best_params_: {'p': 1, 'n_neighbors': 900, 'n_jobs': 4}
    # RESULT rea Fr   best_params_: {'p': 1, 'n_neighbors': 900, 'n_jobs': 4}

    # with not reduced data set:
    # RESULT hosp D31 best_params_: {'p': 2, 'n_neighbors': 900, 'n_jobs': 3}
    #                               {     1                 500  'n_jobs': 90}
    #                               {     1  'n_neighbors': 900  90}
    # RESULT hosp R76 best_params_: {'p': 1, 'n_neighbors': 900, 'n_jobs': 4}
    #                               {     1                 500  'n_jobs': 90}
    # RESULT hosp Fr  best_params_: {'p': 1, 'n_neighbors': 900, 'n_jobs': 4}
    #                               {     1                 500  'n_jobs': 90}
    # RESULT rea D31  best_params_: {'p': 1, 'n_neighbors': 500, 'n_jobs': 3}
    #                               {     1                 500  'n_jobs': 90}
    # RESULT rea R76  best_params_: {'p': 1, 'n_neighbors': 900, 'n_jobs': 4}
    #                               {     1                 500  'n_jobs': 90}
    # RESULT rea Fr   best_params_: {'p': 1, 'n_neighbors': 900, 'n_jobs': 4}
    #                               {     1                 500  'n_jobs': 90}
    #                               {     1  'n_neighbors': 900  90}

    # Search
    if model is not None:
        sh = HalvingRandomSearchCV(model, parameters,
                                min_resources='smallest').fit(X, y)  # min_resources='exhaust'
    else:
        sh = None

    #for param in rnd_search.get_params().keys():
    #    print(param)
    #print('cv_results_\n', sh.cv_results_)
    #print('candidates\n', sh.best_estimator_)
    # print('best_params_:\n', sh.best_params_)
    # print('scorer_:\n', sh.scorer_)

    del df, sel_mode, sel_target, dfm, model, parameters, X, y
    return {'best_param': sh.best_params_, 'scorer': sh.scorer_}

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Search for the best hyper parameter tuning over the various modeling
#  methods, targets and areas.

def run_list_hyperparameters_tuning(df_liste=[], sel_mode='', sel_target=''):
    """
    DESCRIPTION
        Display information related to lists of selection
         (dataframe, model and target) for hyper-parameter tuning.
    INPUT
        df_liste  is a liste of dataframes to work on
                  with structure for targets and models
                  [ [tgt1_model1, tgt1_model2 ...],
                    [tgt2_model1, tgt2_model2 ...] ]
        sel_mode  is the name of the selected modeling method:
                  models_list = ['LinearRegression', 'Ridge', 
                  'NeighborsRegressor_distance']
        sel_target  is the name of the parameters used as answer.
    OUTPUT
        overall_results_dict  is a dictionary with all result per
                              model and target
                {target1: {model1: result1, model2: result2, ...},
                 target2: {...}, ... }
    """
    overall_results_dict = {}

    for t, target in enumerate (targets_list):
        overall_results_dict[target] = {}
        for m, model in enumerate (models_list):
            print("\n>> Target:", target, "- Model:", model)
            start = time.perf_counter()
            result_dict = search_hyperparameters_tuning(
                            df_list[t][m],
                            sel_mode=model,
                            sel_target=target)
            # store the result
            overall_results_dict[target][model] = result_dict

            elapsed = time.perf_counter() - start
            print(">> Time:", elapsed)
            print(">> Result:", result_dict)

    return overall_results_dict

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Search for the best hyper parameter tuning over the various modeling
#  methods, targets and areas.

# Search at department level

targets_list = ['hosp', 'rea']

models_list = ['LinearRegression', 'Ridge',
               'NeighborsRegressor_distance']
df_list = [
    [df_D31_hosp_LinReg, df_D31_hosp_LinRid, df_D31_hosp_NeiReg],\
    [df_D31_rea_LinReg, df_D31_rea_LinRid, df_D31_rea_NeiReg] ]
# use the data set specially refined for the model and target.

hyperparamtuning_results_D31 = run_list_hyperparameters_tuning(
    df_liste=df_list, sel_mode=models_list, sel_target=targets_list)

# RESULT for department D31
>> Target: hosp - Model: LinearRegression
>> Time: 0.11591849999967963
>> Result: {'best_param': {'positive': True, 'fit_intercept': False}, 'scorer': <function _passthrough_scorer at 0x000001E5195D4318>}

>> Target: hosp - Model: Ridge
>> Time: 1.7555947999999262
>> Result: {'best_param': {'tol': 10, 'positive': False, 'max_iter': 75, 'fit_intercept': True, 'alpha': 20}, 'scorer': <function _passthrough_scorer at 0x000001E5195D4318>}
>> with warning messages

>> Target: hosp - Model: NeighborsRegressor_distance
> Time: 25.86618610000005
>> Result: {'best_param': {'p': 2, 'n_neighbors': 700, 'n_jobs': 10}, 'scorer': <function _passthrough_scorer at 0x000001E5195D4318>}
>> with warning messages

>> Target: rea - Model: LinearRegression
>> Time: 0.10736890000043786
>> Result: {'best_param': {'positive': True, 'fit_intercept': False}, 'scorer': <function _passthrough_scorer at 0x000001E5195D4318>}

>> Target: rea - Model: Ridge
>> Time: 1.846445200000744
>> Result: {'best_param': {'tol': 1e-05, 'positive': True, 'max_iter': 800, 'fit_intercept': True, 'alpha': 10}, 'scorer': <function _passthrough_scorer at 0x000001E5195D4318>}
>> with warning messages

>> Target: rea - Model: NeighborsRegressor_distance
>> Time: 22.611654600000293
>> Result: {'best_param': {'p': 2, 'n_neighbors': 900, 'n_jobs': 1}, 'scorer': <function _passthrough_scorer at 0x000001E5195D4318>}
>> with warning messages


# RESUME
Model: LinearRegression
hosp    Result: {'best_param': {'positive': True, 'fit_intercept': False}, ...}
rea     Result: {'best_param': {'positive': True, 'fit_intercept': False}, ...}
>> common settings

Model: Ridge
hosp    Result: {'best_param': {'tol':    10, 'positive': False,  'max_iter': 75, 'fit_intercept': True, 'alpha': 20}, ...}
rea     Result: {'best_param': {'tol': 1e-05, 'positive': True,  'max_iter': 800, 'fit_intercept': True, 'alpha': 10}, ...}
>> Different settings

Model: NeighborsRegressor_distance
hosp    Result: {'best_param': {'p': 2, 'n_neighbors': 700, 'n_jobs': 10}, ...}
rea     Result: {'best_param': {'p': 2, 'n_neighbors': 900,  'n_jobs': 1}, ...}
>> Rather close settings

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Search for the best hyper parameter tuning over the various modeling
#  methods, targets and areas.

# Search at region level

targets_list = ['hosp', 'rea']

models_list = ['LinearRegression', 'Ridge',
               'NeighborsRegressor_distance']
df_list = [
    [df_R76_hosp_LinReg, df_R76_hosp_LinRid, df_R76_hosp_NeiReg],\
    [df_R76_rea_LinReg, df_R76_rea_LinRid, df_R76_rea_NeiReg] ]
# use the data set specially refined for the model and target.

# hyperparamtuning_results_R76 = run_list_hyperparameters_tuning(
#    df_liste=df_list, sel_mode=models_list, sel_target=targets_list)

# No computation at France level because it takes a long time 
#  for the Region. I'm gonna proceed differently.

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Neighbors Regressor
# Define function to allows studying the one by one hyper parameter
#  tuning study and display related score.

def return_df(dfs, target, area):
    """
    DESCRIPTION
        Return the dataframe in accordance wit the target and
        the area.
    INPUT
        dfs  is a dictionary of the dataframe customized and
                refined by area.
        target  is the name (str) of the parameter used as
                answer for modeling.
        area  is the name (str) of the considered area:
                Department 31, Region 76 or France
    OUTPUT
        sel_def = dataframe identified
    """

    try:
        sel_def = dfs[target][area]
    except:
        print('Unable to get the appropriate refined df.')
        sel_def = None

    return sel_def


def param_study_NeighborReg(dfs, target, area):
    """
    DESCRIPTION

    INPUT
        dfs  is a dictionary of the dataframe customized and
                refined by area.
        target  is the name (str) of the parameter used as
                answer for modeling.
        area  is the name (str) of the considered area:
                Department 31, Region 76 or France
    OUTPUT
        display scatter graph of score while changing value  
        of their input tuning parameters.
    """
    m_name = "NeighborsRegressor"

    n_neighbors = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]  #,\
            # 30, 40, 50, 100, 200, 300, 400, 500, 600, 700,\
            # 800, 900, 1000]
    p_values = [1, 2]
    n_jobs = [1, 2, 3, 4, 5, 6, 10, 20, 30, 40, 50, 60, 70,\
                80, 90, 100]

    df_hosp1 = pd.DataFrame([])
    df_hosp2 = pd.DataFrame([])
    df_hosp3 = pd.DataFrame([])

    df = return_df(dfs, target, area)

    X, y = get_X_y(df, target)
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=0.3,
                                    random_state=42)

    scores1, scores2 = [], []
    for n in n_neighbors:
        model = neighbors.KNeighborsRegressor(weights='distance',
                n_neighbors=n,
                p=1,
                n_jobs=None)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = neighbors.KNeighborsRegressor(weights='distance',
                n_neighbors=n,
                p=2,
                n_jobs=None)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    df_hosp1['n'] = n_neighbors
    df_hosp1['score1'] = scores1
    df_hosp1['score2'] = scores2

    scores1, scores2 = [], []
    for p_value in p_values:
        model = neighbors.KNeighborsRegressor(weights='distance',
                n_neighbors=4,
                p=p_value,
                n_jobs=None)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = neighbors.KNeighborsRegressor(weights='distance',
                n_neighbors=5,
                p=p_value,
                n_jobs=None)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    df_hosp2['n'] = p_values
    df_hosp2['score1'] = scores1
    df_hosp2['score2'] = scores2

    scores1, scores2 = [], []
    for n in n_jobs:
        model = neighbors.KNeighborsRegressor(weights='distance',
                n_neighbors=4,
                p=1,
                n_jobs=n)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = neighbors.KNeighborsRegressor(weights='distance',
                n_neighbors=5,
                p=2,
                n_jobs=n)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    df_hosp3['n'] = n_jobs
    df_hosp3['score1'] = scores1
    df_hosp3['score2'] = scores2

    # plot result
    title_base = "Hyper-parameters tuning on " +m_name+" for target"
    title = title_base + " '" + target + "' (" + area + ")"

    plt.figure(figsize=(20, 5))
    plt.title(title)

    # Plot a graph for every label of the x list
    plt.subplot(1, 3, 1)
    #plt = fig.add_subplot(111)
    x = df_hosp1['n']
    y = df_hosp1['score1']
    y2 = df_hosp1['score2']
    plt.scatter(x, y, c='b', marker='o', label='p=1')
    plt.scatter(x, y2, c='g', marker='s', label='p=2')
    plt.title('')
    plt.xlabel('n_neighbors')
    plt.ylabel('score')
    plt.legend(loc='upper right')

    plt.subplot(1, 3, 2)
    x = df_hosp2['n']
    y = df_hosp2['score1']
    y2 = df_hosp2['score2']
    plt.scatter(x, y, c='b', marker='o', label='neighbors=4')
    plt.scatter(x, y2, c='g', marker='s', label='neighbors=5')
    plt.title('')
    plt.xlabel('n_p')
    plt.ylabel('score')
    plt.legend(loc='upper right')

    plt.subplot(1, 3, 3)
    x = df_hosp3['n']
    y = df_hosp3['score1']
    y2 = df_hosp3['score2']
    plt.scatter(x, y, c='b', marker='o', label='p=1 / neighbors=4')
    plt.scatter(x, y2, c='g', marker='s', label='p=2 / neighbors=5')
    plt.title('')
    plt.xlabel('n_jobs')
    plt.ylabel('score')
    plt.legend(loc='center left')

    plt.suptitle(title)
    plt.show()

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Define function to allows studying the one by one hyper parameter
#  tuning study and display related score.

def param_study_LinearReg(dfs, target, area):
    """
    DESCRIPTION

    INPUT
        dfs  is a dictionary of the dataframe customized and
                refined by area.
        target  is the name (str) of the parameter used as
                answer for modeling.
        area  is the name (str) of the considered area:
                Department 31, Region 76 or France
    OUTPUT
        display scatter graph of score while changing value  
        of their input tuning parameters.
    """
    m_name = "LinearRegressor"

    fit_values = [True, False]
    positive_values = [True, False]

    df_hosp1 = pd.DataFrame([])
    df_hosp2 = pd.DataFrame([])
    df_hosp3 = pd.DataFrame([])

    df = return_df(dfs, target, area)

    X, y = get_X_y(df, target)
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=0.3,
                                    random_state=42)

    scores1, scores2 = [], []
    for fit_value in fit_values:
        model = LinearRegression(fit_intercept=fit_value,
                                 positive=True)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = LinearRegression(fit_intercept=fit_value,
                                 positive=False)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    df_hosp1['n'] = list(map(int, fit_values))  # convert boolean to int
    df_hosp1['score1'] = scores1
    df_hosp1['score2'] = scores2

    scores1, scores2 = [], []
    for positive_value in positive_values:
        model = LinearRegression(fit_intercept=True,
                                 positive=positive_value)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = LinearRegression(fit_intercept=False,
                                 positive=positive_value)
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    # convert boolean to int
    df_hosp2['n'] = list(map(int, positive_values))
    
    df_hosp2['score1'] = scores1
    df_hosp2['score2'] = scores2


    # plot result
    title_base = "Hyper-parameters tuning on " +m_name+" for target"
    title = title_base + " '" + target + "' (" + area + ")"

    plt.figure(figsize=(20, 5))
    plt.title(title)

    # Plot a graph for every label of the x list
    plt.subplot(1, 2, 1)
    #plt = fig.add_subplot(111)
    x = df_hosp1['n']
    y = df_hosp1['score1']
    y2 = df_hosp1['score2']
    plt.scatter(x, y, c='b', marker='o', label='positive=True')
    plt.scatter(x, y2, c='g', marker='s', label='positive=False')
    plt.title('')
    plt.xlabel('fit')
    plt.ylabel('score')
    plt.legend(loc='upper center')

    plt.subplot(1, 2, 2)
    x = df_hosp2['n']
    y = df_hosp2['score1']
    y2 = df_hosp2['score2']
    plt.scatter(x, y, c='b', marker='o', label='fit_intercept=True')
    plt.scatter(x, y2, c='g', marker='s', label='fit_intercept=False')
    plt.title('')
    plt.xlabel('positive')
    plt.ylabel('score')
    plt.legend(loc='upper right')

    plt.suptitle(title)
    plt.show()

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Define function to allows studying the one by one hyper parameter
#  tuning study and display related score.

def ident_log_scale(lst):
    """
    DESCRIPTION 
        identify if values of a list should be plot on a log scale or
         not.
    INPUT
        lst  is a list of value
    OUTPUT
        log_sts  is a boolean: True=log, False=not log
    """
    log_sts = False
    for i in range(len(lst)-1):
        if lst[i+1] != 0:
            tx = abs(lst[i]/lst[i+1])
            if (tx >= 10) or (tx <= 0.1):
                log_sts = True
    del lst
    return log_sts


def param_study_LinearRidge(dfs, target, area):
    """
    DESCRIPTION

    INPUT
        dfs  is a dictionary of the dataframe customized and
                refined by area.
        target  is the name (str) of the parameter used as
                answer for modeling.
        area  is the name (str) of the considered area:
                Department 31, Region 76 or France
    OUTPUT
        display scatter graph of score while changing value  
        of their input tuning parameters.
    """
    m_name = "LinearRidge"

    fit_intercept_values = [True, False]
    positive_values = [True, False]
    alpha_values = [0.1, 0.5, 1, 5, 10, 20]
    max_iter_values = [10, 25, 50, 75, 100, 500, 600, 700, 800, 900, 1000]
    tol_values = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e-0, 10, 100]

    df_hosp1 = pd.DataFrame([])
    df_hosp2 = pd.DataFrame([])
    df_hosp3 = pd.DataFrame([])
    df_hosp4 = pd.DataFrame([])
    df_hosp5 = pd.DataFrame([])

    df = return_df(dfs, target, area)

    X, y = get_X_y(df, target)
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=0.3,
                                    random_state=42)

    scores1, scores2 = [], []
    for fit_intercept_value in fit_intercept_values:
        model = Ridge(alpha=1,
                    fit_intercept=fit_intercept_value,
                    positive=True,
                    max_iter=900,
                    tol=1
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = Ridge(alpha=1,
                    fit_intercept=fit_intercept_value,
                    positive=False,
                    max_iter=900,
                    tol=1
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    # convert boolean to int
    df_hosp1['n'] = list(map(int, fit_intercept_values))
    df_hosp1['score1'] = scores1
    df_hosp1['score2'] = scores2

    scores1, scores2 = [], []
    for alpha_value in alpha_values:
        model = Ridge(alpha=alpha_value,
                    fit_intercept=True,
                    positive=True,
                    max_iter=900,
                    tol=1
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = Ridge(alpha=alpha_value,
                    fit_intercept=True,
                    positive=False,
                    max_iter=900,
                    tol=1
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    df_hosp2['n'] = alpha_values
    df_hosp2['score1'] = scores1
    df_hosp2['score2'] = scores2

    scores1, scores2 = [], []
    for max_iter_value in max_iter_values:
        model = Ridge(alpha=1,
                    fit_intercept=True,
                    positive=True,
                    max_iter=max_iter_value,
                    tol=1
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = Ridge(alpha=1,
                    fit_intercept=True,
                    positive=False,
                    max_iter=max_iter_value,
                    tol=1
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    df_hosp3['n'] = max_iter_values
    df_hosp3['score1'] = scores1
    df_hosp3['score2'] = scores2

    scores1, scores2 = [], []
    for tol_value in tol_values:
        model = Ridge(alpha=1,
                    fit_intercept=True,
                    positive=True,
                    max_iter=900,
                    tol=tol_value
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = Ridge(alpha=1,
                    fit_intercept=True,
                    positive=False,
                    max_iter=900,
                    tol=tol_value
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    df_hosp4['n'] = tol_values
    df_hosp4['score1'] = scores1
    df_hosp4['score2'] = scores2

    scores1, scores2 = [], []
    for positive_value in positive_values:
        model = Ridge(alpha=1,
                    fit_intercept=True,
                    positive=positive_value,
                    max_iter=900,
                    tol=1
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores1.append(score_value)
        model = Ridge(alpha=1,
                    fit_intercept=False,
                    positive=positive_value,
                    max_iter=900,
                    tol=1
                    )
        model.fit(X_train, y_train)
        try:
            score_value = model.score(X_test, y_test)
        except:
            score_value = np.nan
        scores2.append(score_value)
    # convert boolean to int
    df_hosp5['n'] = list(map(int, positive_values))
    df_hosp5['score1'] = scores1
    df_hosp5['score2'] = scores2

    # plot result
    title_base = "Hyper-parameters tuning on " +m_name+" for target"
    title = title_base + " '" + target + "' (" + area + ")"

    plt.figure(figsize=(20, 5))
    plt.title(title)

    # Plot a graph for every label of the x list
    plt.subplot(1, 5, 1)
    #plt = fig.add_subplot(111)
    x = df_hosp1['n']
    y = df_hosp1['score1']
    y2 = df_hosp1['score2']
    plt.scatter(x, y, c='b', marker='o', label='positive=True')
    plt.scatter(x, y2, c='g', marker='s', label='positive=False')
    x_log = ident_log_scale(x)
    y_log = ident_log_scale(y)
    y2_log = ident_log_scale(y2)
    if x_log:
        plt.xscale("log")
    if y_log or y2_log:
        plt.yscale("log")
    plt.title('')
    plt.xlabel('fit_intercept')
    plt.ylabel('score')
    plt.legend(loc='upper center')

    plt.subplot(1, 5, 2)
    x = df_hosp2['n']
    y = df_hosp2['score1']
    y2 = df_hosp2['score2']
    plt.scatter(x, y, c='b', marker='o', label='positive=True')
    plt.scatter(x, y2, c='g', marker='s', label='positive=False')
    x_log = ident_log_scale(x)
    y_log = ident_log_scale(y)
    y2_log = ident_log_scale(y2)
    if x_log:
        plt.xscale("log")
    if y_log or y2_log:
        plt.yscale("log")
    plt.title('')
    plt.xlabel('alpha')
    plt.ylabel('score')
    plt.legend(loc='upper right')

    plt.subplot(1, 5, 3)
    x = df_hosp3['n']
    y = df_hosp3['score1']
    y2 = df_hosp3['score2']
    plt.scatter(x, y, c='b', marker='o', label='positive=True')
    plt.scatter(x, y2, c='g', marker='s', label='positive=False')
    x_log = ident_log_scale(x)
    y_log = ident_log_scale(y)
    y2_log = ident_log_scale(y2)
    if x_log:
        plt.xscale("log")
    if y_log or y2_log:
        plt.yscale("log")
    plt.title('')
    plt.xlabel('max_iter')
    plt.ylabel('score')
    plt.legend(loc='center left')

    plt.subplot(1, 5, 4)
    x = df_hosp4['n']
    y = df_hosp4['score1']
    y2 = df_hosp4['score2']
    plt.scatter(x, y, c='b', marker='o', label='positive=True')
    plt.scatter(x, y2, c='g', marker='s', label='positive=False')
    x_log = ident_log_scale(x)
    y_log = ident_log_scale(y)
    y2_log = ident_log_scale(y2)
    if x_log:
        plt.xscale("log")
    if y_log or y2_log:
        plt.yscale("log")
    plt.title('')
    plt.xlabel('tol')
    plt.ylabel('score')
    plt.legend(loc='center left')

    plt.subplot(1, 5, 5)
    x = df_hosp5['n']
    y = df_hosp5['score1']
    y2 = df_hosp5['score2']
    plt.scatter(x, y, c='b', marker='o', label='fit_intercept=True')
    plt.scatter(x, y2, c='g', marker='s', label='fit_intercept=False')
    x_log = ident_log_scale(x)
    y_log = ident_log_scale(y)
    y2_log = ident_log_scale(y2)
    if x_log:
        plt.xscale("log")
    if y_log or y2_log:
        plt.yscale("log")
    plt.title('')
    plt.xlabel('positive')
    plt.ylabel('score')
    plt.legend(loc='center left')

    plt.suptitle(title)
    plt.show()

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Neighbors Regressor
# Build the dictionary of appropriate refined dataframes

dfs = {'hosp': {'Department 31': df_D31_hosp_NeiReg,
                'Region 76': df_R76_hosp_NeiReg,
                'France': df_Fr_hosp_NeiReg
               },
       'rea': {'Department 31': df_D31_rea_NeiReg,
                'Region 76': df_R76_rea_NeiReg,
                'France': df_Fr_rea_NeiReg
              }
      }

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Neighbors Regressor
# Apply for target 'hosp' at department level

target = 'hosp'
area = 'Department 31'
param_study_NeighborReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Neighbors Regressor
# Apply for target 'hosp' at region level

target = 'hosp'
area = 'Region 76'
param_study_NeighborReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Neighbors Regressor
# Apply for target 'hosp' at Country level

target = 'hosp'
area = 'France'
param_study_NeighborReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Neighbors Regressor
# Apply for target 'rea' at department level

target = 'rea'
area = 'Department 31'
param_study_NeighborReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Neighbors Regressor
# Apply for target 'rea' at Region level

target = 'rea'
area = 'Region 76'
param_study_NeighborReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Neighbors Regressor
# Apply for target 'rea' at Region level

target = 'rea'
area = 'France'
# param_study_NeighborReg(dfs, target, area)
# Cancelled because it run 26min 0.5s without display neither plot nor error or warning message.

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'hosp' at department level

target = 'hosp'
area = 'Department 31'
param_study_LinearReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'hosp' at region level

target = 'hosp'
area = 'Region 76'
param_study_LinearReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'hosp' at country level

target = 'hosp'
area = 'France'
param_study_LinearReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'rea' at department level

target = 'rea'
area = 'Department 31'
param_study_LinearReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'rea' at region level

target = 'rea'
area = 'Region 76'
param_study_LinearReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'rea' at country level

target = 'rea'
area = 'France'
param_study_LinearReg(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'hosp' at department level

target = 'hosp'
area = 'Department 31'
param_study_LinearRidge(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'hosp' at region level

target = 'hosp'
area = 'Region 76'
param_study_LinearRidge(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'hosp' at country level

target = 'hosp'
area = 'France'
param_study_LinearRidge(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'rea' at department level

target = 'rea'
area = 'Department 31'
param_study_LinearRidge(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'rea' at region level

target = 'rea'
area = 'Region 76'
param_study_LinearRidge(dfs, target, area)

In [ ]:
# METHODOLOGY - Implementation - Hyperparameter tuning
# ----------------------------------------------------------------------========
# Proceed for searching searching method for Linear Regressor
# Apply for target 'hosp' at country level

target = 'rea'
area = 'France'
param_study_LinearRidge(dfs, target, area)

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
# Create a function to vizualize result on plots

def display_results(df, x_name, y_name, graph_title,
                    subtitle1='', subtitle2=''):

    plt.figure(figsize=(20, 18))

    plt.subplot(3, 1, 1)
    x = df[x_name]
    y = df[y_name]
    plt.scatter(x, y, c='g', marker='o')
    plt.title('(' + subtitle1 + ' ; ' + subtitle2 + ')')
    plt.xlabel(x_name)
    plt.ylabel(y_name)
    plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)

    plt.subplot(3, 1, 2)
    x = df[x_name]
    abs_error = np.subtract(y, x)
    rel_error = np.divide(abs_error, x)
    mini = np.min(rel_error)
    maxi = np.max(rel_error)
    mean = np.mean(rel_error)
    quantile = np.quantile(rel_error, 0.95)
    plt.scatter(x, rel_error, c='r', marker='s')
    msg1 = str(mini)[:5] + '% (min) ; '
    msg2 = str(maxi)[:5] + '% (max) ; '
    msg3 = str(mean)[:5] + '% (mean) ;'
    msg4 = str(quantile)[:5] + '% (95%-quantile)'
    msgA = 'Rel. error: ' + msg1 + msg2 + msg3 + msg4
    mini = np.min(abs_error)
    maxi = np.max(abs_error)
    mean = np.mean(abs_error)
    quantile = np.quantile(abs_error, 0.95)
    msg1 = str(mini)[:5] + ' (min) ; '
    msg2 = str(maxi)[:5] + ' (max) ; '
    msg3 = str(mean)[:5] + ' (mean) ;'
    msg4 = str(quantile)[:5] + ' (95%-quantile)'
    msgB = 'Abs. error: ' + msg1 + msg2 + msg3 + msg4
    plt.title(msgA + ' / '+ msgB)
    plt.xlabel(x_name)
    plt.ylabel('relative error (%)')
    plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)
    #plt.yscale(value="log")

    try:
        plt.subplot(3, 1, 3)
        bins_nb = int(pd.DataFrame(rel_error).nunique() / 15)
        plt.hist(rel_error, bins=bins_nb, density=True)
        plt.title("Distribution of the relative error values" + sb)
        plt.xlabel('relative error (%)')
        plt.ylabel('density')
    except:
        print("Distribution ploting failure")

    plt.suptitle(graph_title, y=0.92)
    plt.show()

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
# Define the function to build, train, fit and use the model  

def main(df, target, title):

    df_mdl = df.copy(deep=True)

    # split data set
    testrate = 0.3
    X, y = get_X_y(df_mdl, target)
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=testrate,
                                    random_state=42)

    # build model
    model = neighbors.KNeighborsRegressor(weights='distance',
                                          p=1,
                                          n_neighbors=4)
    #                                     n_neighbors=500,
    #                                     n_jobs=4)
        
    # train model
    model.fit(X_train, y_train)
    
    # predict on test data
    y_pred = model.predict(X_test)

   # Scoring
    try:
        r2_score_value = r2_score(y, model.predict(X))
    except:
        r2_score_value = None
    try:
        score_value = model.score(X_test, y_test)
    except:
        score_value = None
    # print('r2 score     :', r2_score_value)
    # print("Model's score:", score_value)

    # display results
    display_results(pd.DataFrame({'y_test': y_test.tolist(), 'y_pred': y_pred.tolist()}),
                    'y_test', 'y_pred',
                    'Predictions vs True values - ' + title,
                    subtitle1='R2 score = ' + str(r2_score_value), subtitle2='score = ' + str(score_value))

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main(df_D31_hosp_NeiReg, 'hosp', "Number of hospitalization in the department 31 (Neighbors Regressor method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main(df_D31_rea_NeiReg, 'rea', "Number of people in resuscitation in the department 31 (Neighbors Regressor method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main(df_R76_hosp_NeiReg, 'hosp', "Number of hospitalization in the Region 76 (Neighbors Regressor method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main(df_R76_rea_NeiReg, 'rea', "Number of people in resuscitation in the Region 76 (Neighbors Regressor method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main(df_Fr_hosp_NeiReg, 'hosp', "Number of hospitalization in France (Neighbors Regressor method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main(df_Fr_rea_NeiReg, 'rea', "Number of people in resuscitation in France (Neighbors Regressor method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
# Define the function to build, train, fit and use the model
# Compare with Linear Regression model

def main2(df, target, title):

    df_mdl = df.copy(deep=True)

    # split data set
    testrate = 0.3
    X, y = get_X_y(df_mdl, target)
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=testrate,
                                    random_state=42)

    # build model
    model = LinearRegression(positive=True, fit_intercept=False)
        
    # train model
    model.fit(X_train, y_train)
    
    # predict on test data
    y_pred = model.predict(X_test)

   # Scoring
    try:
        r2_score_value = r2_score(y, model.predict(X))
    except:
        r2_score_value = None
    try:
        score_value = model.score(X_test, y_test)
    except:
        score_value = None
    # print('r2 score     :', r2_score_value)
    # print("Model's score:", score_value)

    # display results
    display_results(pd.DataFrame({'y_test': y_test.tolist(), 'y_pred': y_pred.tolist()}),
                    'y_test', 'y_pred',
                    'Predictions vs True values - ' + title,
                    subtitle1='R2 score = ' + str(r2_score_value), subtitle2='score = ' + str(score_value))

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main2(df_D31_hosp_LinReg, 'hosp', "Number of hospitalization in the department 31 (LinearRegression method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main2(df_D31_rea_LinReg, 'rea', "Number of people in resuscitation in the department 31 (LinearRegression method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main2(df_R76_hosp_LinReg, 'hosp', "Number of hospitalization in the Region 76 (LinearRegression method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main2(df_R76_rea_LinReg, 'rea', "Number of people in resuscitation in the Region 76 (LinearRegression method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main2(df_Fr_hosp_LinReg, 'hosp', "Number of hospitalization in France (LinearRegression method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main2(df_Fr_rea_LinReg, 'rea', "Number of people in resuscitation in France (LinearRegression method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
# Define the function to build, train, fit and use the model
# Compare with Linear Ridge model

def main3(df, target, title):

    df_mdl = df.copy(deep=True)

    # split data set
    testrate = 0.3
    X, y = get_X_y(df_mdl, target)
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=testrate,
                                    random_state=42)

    # build model
    model = Ridge(fit_intercept=True,
                  alpha=20,
                  positive=False,
                  max_iter=900,
                  tol=10)

    # train model
    model.fit(X_train, y_train)
    
    # predict on test data
    y_pred = model.predict(X_test)

   # Scoring
    try:
        r2_score_value = r2_score(y, model.predict(X))
    except:
        r2_score_value = None
    try:
        score_value = model.score(X_test, y_test)
    except:
        score_value = None
    # print('r2 score     :', r2_score_value)
    # print("Model's score:", score_value)

    # display results
    display_results(pd.DataFrame({'y_test': y_test.tolist(), 'y_pred': y_pred.tolist()}),
                    'y_test', 'y_pred',
                    'Predictions vs True values - ' + title,
                    subtitle1='R2 score = ' + str(r2_score_value), subtitle2='score = ' + str(score_value))

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main3(df_D31_hosp_LinRid, 'hosp', "Number of hospitalization in the department 31 (Linear Ridge method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main3(df_D31_rea_LinRid, 'rea', "Number of people in resuscitation in the department 31 (Linear Ridge method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main3(df_R76_hosp_LinRid, 'hosp', "Number of hospitalization in the Region 76 (Linear Ridge method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main3(df_R76_rea_LinRid, 'rea', "Number of people in resuscitation in the Region 76 (Linear Ridge method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main3(df_Fr_hosp_LinRid, 'hosp', "Number of hospitalization in France (Linear Ridge method)")

In [ ]:
# METHODOLOGY - Vizualization
# ----------------------------------------------------------------------========
main3(df_Fr_rea_LinRid, 'rea', "Number of people in resuscitation in France (Linear Ridge method)")